# 1교시 - PySpark DataFrame API 종합 과제

---

## 과제 개요

이 과제에서는 실제 데이터를 활용하여 PySpark DataFrame API를 종합적으로 실습합니다.

**사용 데이터:**
- **Tips 데이터셋**: 레스토랑 팁 기록 데이터 (244건)
- **Titanic 데이터셋**: 타이타닉 승객 생존 데이터 (891건)

**학습 목표:**
- 컬럼 선택, 추가, 수정 (`select`, `withColumn`)
- 필터링 (`filter`, `where`, `isin`)
- 조건부 값 설정 (`when`, `otherwise`)
- 정렬, 중복 제거, 제한
- 그룹별 집계 (`groupBy`, `agg`)
- 피벗 테이블 (`pivot`)
- 조인 (`join`)
- NULL 처리 (`dropna`, `fillna`, `coalesce`)
- Window 함수 (순위, 누적합, 이동평균)

---

## 환경 설정

In [ ]:
# -----------------------------------------------------------------------------
# 환경 설정: PySpark 및 데이터 로드
# -----------------------------------------------------------------------------
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, when, count, sum, avg, min, max,
    round as spark_round, expr,
    upper, lower, trim, concat, concat_ws, substring, length,
    to_date, year, month, dayofmonth, datediff, current_date,
    row_number, rank, dense_rank, lag, lead,
    coalesce, countDistinct, first, last,
)
from pyspark.sql.window import Window
import seaborn as sns
import pandas as pd

# SparkSession 생성
spark = SparkSession.builder \
    .appName("PySpark-DataFrame-Exercise") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", 10) \
    .getOrCreate()

# -----------------------------------------------------------------------------
# 실제 데이터 로드: Seaborn 내장 데이터셋
# -----------------------------------------------------------------------------

# Tips 데이터셋: 레스토랑 팁 기록
tips_pdf = sns.load_dataset("tips")
df_tips = spark.createDataFrame(tips_pdf)

# Titanic 데이터셋: 타이타닉 승객 생존 기록
titanic_pdf = sns.load_dataset("titanic")
df_titanic = spark.createDataFrame(titanic_pdf)

print("=" * 60)
print("데이터 로드 완료!")
print("=" * 60)
print(f"\n[Tips 데이터]")
print(f"- 행 수: {df_tips.count()}")
print(f"- 컬럼: {df_tips.columns}")
df_tips.show(5)

print(f"\n[Titanic 데이터]")
print(f"- 행 수: {df_titanic.count()}")
print(f"- 컬럼: {df_titanic.columns}")
df_titanic.show(5)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/21 10:55:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


데이터 로드 완료!

[Tips 데이터]


- 행 수: 244
- 컬럼: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']
+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows


[Titanic 데이터]
- 행 수: 891
- 컬럼: ['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone']
+--------+------+------+----+-----+-----+-------+--------+-----+-----+----------+----+-----------+-----+-----+
|survived|pclass|   sex| age|sibsp|parch|   fare|embarked|class|  who|adult_male|deck|embark_town|alive|alone|
+--------+------+------+----+-----+-----+-------+--------+-----+

26/01/21 10:55:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [8]:
!pip install seaborn setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 19.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 17.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 23.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


---

## Part 1: 데이터 탐색

Tips 데이터의 스키마를 확인하고 기본 통계를 출력하세요.

### 문제 1-1: 스키마 출력

Tips 데이터의 스키마를 출력하세요.

<details>
<summary>힌트</summary>

DataFrame의 `printSchema()` 메서드를 호출하면 됩니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.printSchema()
```

</details>

In [ ]:
# [문제 1-1] 여기에 코드를 작성하세요



### 문제 1-2: 기본 통계 확인

Tips 데이터의 기본 통계(count, mean, stddev, min, max)를 확인하세요.

<details>
<summary>힌트</summary>

`describe()` 메서드는 숫자형 컬럼에 대한 통계를 반환합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.describe().show()
```

</details>

In [ ]:
# [문제 1-2] 여기에 코드를 작성하세요



---

## Part 2: 컬럼 선택 및 조작

### 문제 2-1: 컬럼 선택

Tips 데이터에서 `total_bill`, `tip`, `day` 컬럼만 선택하세요.

<details>
<summary>힌트</summary>

`select("컬럼1", "컬럼2", "컬럼3")` 형태로 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.select("total_bill", "tip", "day").show(5)
```

</details>

In [ ]:
# [문제 2-1] 여기에 코드를 작성하세요



### 문제 2-2: 새 컬럼 추가 (팁 비율)

`tip`을 `total_bill`로 나눈 `tip_rate` 컬럼을 추가하세요.
소수점 2자리로 반올림하세요.

<details>
<summary>힌트</summary>

- `withColumn()`으로 새 컬럼 추가
- `spark_round(값, 자릿수)`로 반올림
- `col("컬럼") / col("컬럼")`으로 나눗셈

</details>

<details>
<summary>모범 답안</summary>

```python
df_with_rate = df_tips.withColumn(
    "tip_rate",
    spark_round(col("tip") / col("total_bill"), 2)
)
df_with_rate.select("total_bill", "tip", "tip_rate").show(5)
```

</details>

In [ ]:
# [문제 2-2] 여기에 코드를 작성하세요



### 문제 2-3: 컬럼 이름 변경

`total_bill` 컬럼을 `bill_amount`로, `tip` 컬럼을 `tip_amount`로 변경하세요.

<details>
<summary>힌트</summary>

`withColumnRenamed("기존이름", "새이름")`을 체이닝합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_renamed = df_tips \
    .withColumnRenamed("total_bill", "bill_amount") \
    .withColumnRenamed("tip", "tip_amount")
df_renamed.show(5)
```

</details>

In [ ]:
# [문제 2-3] 여기에 코드를 작성하세요



### 문제 2-4: 상수 컬럼 추가

모든 행에 `currency` 컬럼을 추가하고 값을 `"USD"`로 설정하세요.

<details>
<summary>힌트</summary>

`lit("값")`을 사용하여 상수값을 Column으로 변환합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_with_currency = df_tips.withColumn("currency", lit("USD"))
df_with_currency.select("total_bill", "currency").show(5)
```

</details>

In [ ]:
# [문제 2-4] 여기에 코드를 작성하세요



---

## Part 3: 필터링

### 문제 3-1: 단순 필터링

Tips 데이터에서 `total_bill`이 20 이상인 행만 필터링하세요.

<details>
<summary>힌트</summary>

`filter(col("컬럼") >= 값)` 또는 `filter("컬럼 >= 값")` 형태로 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.filter(col("total_bill") >= 20).show(5)
# 또는
df_tips.filter("total_bill >= 20").show(5)
```

</details>

In [ ]:
# [문제 3-1] 여기에 코드를 작성하세요



### 문제 3-2: AND 조건 필터링

`total_bill`이 20 이상이고 `tip`이 3 이상인 행만 필터링하세요.

**주의:** AND 조건에서는 각 조건을 괄호로 감싸야 합니다!

<details>
<summary>힌트</summary>

`(조건1) & (조건2)` 형태로 AND 조건을 작성합니다.
괄호를 반드시 사용해야 합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.filter(
    (col("total_bill") >= 20) & (col("tip") >= 3)
).show(5)
# 또는 SQL 스타일
df_tips.filter("total_bill >= 20 AND tip >= 3").show(5)
```

</details>

In [ ]:
# [문제 3-2] 여기에 코드를 작성하세요



### 문제 3-3: OR 조건 필터링

`day`가 "Sat" 또는 "Sun"인 행만 필터링하세요.

<details>
<summary>힌트</summary>

방법 1: `(조건1) | (조건2)` 형태
방법 2: `col("컬럼").isin(값1, 값2)` 형태

</details>

<details>
<summary>모범 답안</summary>

```python
# 방법 1: OR 연산자
df_tips.filter(
    (col("day") == "Sat") | (col("day") == "Sun")
).show(5)

# 방법 2: isin() 사용 (권장)
df_tips.filter(col("day").isin("Sat", "Sun")).show(5)
```

</details>

In [ ]:
# [문제 3-3] 여기에 코드를 작성하세요



### 문제 3-4: 문자열 필터링

Titanic 데이터에서 `embark_town`이 "South"로 시작하는 승객을 필터링하세요.

<details>
<summary>힌트</summary>

`col("컬럼").startswith("문자열")` 메서드를 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_titanic.filter(
    col("embark_town").startswith("South")
).select("survived", "name", "embark_town").show(5)
```

</details>

In [ ]:
# [문제 3-4] 여기에 코드를 작성하세요



---

## Part 4: 조건부 값 설정 (when / otherwise)

### 문제 4-1: 단순 조건 분기

Tips 데이터에 `tip_quality` 컬럼을 추가하세요.
- `tip`이 5 이상이면 "Good"
- 그 외에는 "Normal"

<details>
<summary>힌트</summary>

`when(조건, 값).otherwise(기본값)` 구조를 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_quality = df_tips.withColumn(
    "tip_quality",
    when(col("tip") >= 5, "Good").otherwise("Normal")
)
df_quality.select("tip", "tip_quality").show(10)
```

</details>

In [ ]:
# [문제 4-1] 여기에 코드를 작성하세요



### 문제 4-2: 다중 조건 분기

Tips 데이터에 `bill_category` 컬럼을 추가하세요.
- `total_bill` >= 30: "High"
- `total_bill` >= 20: "Medium"
- 그 외: "Low"

<details>
<summary>힌트</summary>

`when().when().otherwise()` 형태로 체이닝합니다.
첫 번째로 참인 조건에서 멈춥니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_category = df_tips.withColumn(
    "bill_category",
    when(col("total_bill") >= 30, "High")
    .when(col("total_bill") >= 20, "Medium")
    .otherwise("Low")
)
df_category.select("total_bill", "bill_category").show(10)

# 카테고리별 건수 확인
df_category.groupBy("bill_category").count().show()
```

</details>

In [ ]:
# [문제 4-2] 여기에 코드를 작성하세요



### 문제 4-3: 복합 조건 분기

Titanic 데이터에 `priority` 컬럼을 추가하세요.
- 1등석(`pclass` == 1)이면서 여성(`sex` == "female"): "Highest"
- 여성: "High"
- 어린이(`age` < 12): "High"
- 그 외: "Normal"

<details>
<summary>힌트</summary>

`when()` 안에서 AND 조건은 `&`를 사용하고 괄호로 감쌉니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_priority = df_titanic.withColumn(
    "priority",
    when((col("pclass") == 1) & (col("sex") == "female"), "Highest")
    .when(col("sex") == "female", "High")
    .when(col("age") < 12, "High")
    .otherwise("Normal")
)
df_priority.select("pclass", "sex", "age", "priority").show(10)

# 우선순위별 생존율 확인
df_priority.groupBy("priority").agg(
    count("*").alias("total"),
    sum("survived").alias("survived"),
    spark_round(avg("survived"), 2).alias("survival_rate")
).show()
```

</details>

In [ ]:
# [문제 4-3] 여기에 코드를 작성하세요



---

## Part 5: 정렬, 중복 제거, 제한

### 문제 5-1: 정렬

Tips 데이터를 `total_bill` 내림차순으로 정렬하고 상위 10개를 출력하세요.

<details>
<summary>힌트</summary>

`orderBy(col("컬럼").desc())`로 내림차순 정렬합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.orderBy(col("total_bill").desc()).show(10)
```

</details>

In [4]:
# [문제 5-1] 여기에 코드를 작성하세요
df_tips.orderBy(col("total_bill").desc()).show(10)


+----------+----+------+------+----+------+----+
|total_bill| tip|   sex|smoker| day|  time|size|
+----------+----+------+------+----+------+----+
|     50.81|10.0|  Male|   Yes| Sat|Dinner|   3|
|     48.33| 9.0|  Male|    No| Sat|Dinner|   4|
|     48.27|6.73|  Male|    No| Sat|Dinner|   4|
|     48.17| 5.0|  Male|    No| Sun|Dinner|   6|
|     45.35| 3.5|  Male|   Yes| Sun|Dinner|   3|
|      44.3| 2.5|Female|   Yes| Sat|Dinner|   3|
|     43.11| 5.0|Female|   Yes|Thur| Lunch|   4|
|     41.19| 5.0|  Male|    No|Thur| Lunch|   5|
|     40.55| 3.0|  Male|   Yes| Sun|Dinner|   2|
|     40.17|4.73|  Male|   Yes| Fri|Dinner|   4|
+----------+----+------+------+----+------+----+
only showing top 10 rows



### 문제 5-2: 여러 컬럼 정렬

Tips 데이터를 `day` 오름차순, `total_bill` 내림차순으로 정렬하세요.

<details>
<summary>힌트</summary>

`orderBy(col("컬럼1").asc(), col("컬럼2").desc())` 형태로 여러 컬럼 정렬을 지정합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.orderBy(
    col("day").asc(),
    col("total_bill").desc()
).show(15)
```

</details>

In [ ]:
# [문제 5-2] 여기에 코드를 작성하세요
df_tips.orderBy(
    col("day").asc(),   
    col("total_bill").desc()      
).show(10)


+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     40.17|4.73|  Male|   Yes|Fri|Dinner|   4|
|     28.97| 3.0|  Male|   Yes|Fri|Dinner|   2|
|     27.28| 4.0|  Male|   Yes|Fri|Dinner|   2|
|     22.75|3.25|Female|    No|Fri|Dinner|   2|
|     22.49| 3.5|  Male|    No|Fri|Dinner|   2|
|     21.01| 3.0|  Male|   Yes|Fri|Dinner|   2|
|     16.32| 4.3|Female|   Yes|Fri|Dinner|   2|
|     16.27| 2.5|Female|   Yes|Fri| Lunch|   2|
|     15.98| 3.0|Female|    No|Fri| Lunch|   3|
|     15.38| 3.0|Female|   Yes|Fri|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 10 rows



### 문제 5-3: 고유값 추출

Tips 데이터에서 `day`의 고유값 목록을 추출하세요.

<details>
<summary>힌트</summary>

`select("컬럼").distinct()`를 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.select("day").distinct().show()
```

</details>

In [7]:
# [문제 5-3] 여기에 코드를 작성하세요
df_tips.select("day").distinct().show()


+----+
| day|
+----+
| Sun|
| Sat|
|Thur|
| Fri|
+----+



### 문제 5-4: Top N 추출

Tips 데이터에서 `tip`이 가장 높은 5건을 추출하세요.

<details>
<summary>힌트</summary>

`orderBy().limit()` 조합을 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.orderBy(col("tip").desc()).limit(5).show()
```

</details>

In [8]:
# [문제 5-4] 여기에 코드를 작성하세요
df_tips.orderBy(col("tip").desc()).limit(5).show()


+----------+----+----+------+----+------+----+
|total_bill| tip| sex|smoker| day|  time|size|
+----------+----+----+------+----+------+----+
|     50.81|10.0|Male|   Yes| Sat|Dinner|   3|
|     48.33| 9.0|Male|    No| Sat|Dinner|   4|
|     39.42|7.58|Male|    No| Sat|Dinner|   4|
|     48.27|6.73|Male|    No| Sat|Dinner|   4|
|      34.3| 6.7|Male|    No|Thur| Lunch|   6|
+----------+----+----+------+----+------+----+



---

## Part 6: 그룹별 집계

### 문제 6-1: 단일 컬럼 그룹 집계

Tips 데이터를 `day`별로 그룹화하여 다음을 계산하세요:
- 건수 (`count`)
- 평균 팁 (`avg_tip`)
- 총 매출 (`total_bill_sum`)

<details>
<summary>힌트</summary>

`groupBy("컬럼").agg(집계함수.alias("별칭"), ...)`

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.groupBy("day").agg(
    count("*").alias("count"),
    spark_round(avg("tip"), 2).alias("avg_tip"),
    sum("total_bill").alias("total_bill_sum")
).orderBy("day").show()
```

</details>

In [11]:
# [문제 6-1] 여기에 코드를 작성하세요
total_stats = df_tips.groupBy("day").agg(
    count("*").alias("건수"),                    
    spark_round(avg("tip"),2).alias("평균팁"),               # 급여 평균
    sum("total_bill").alias("총매출"),               # 최소 급여

).show()


+----+----+------+------------------+
| day|건수|평균팁|            총매출|
+----+----+------+------------------+
| Sun|  76|  3.26|1627.1599999999996|
| Sat|  87|  2.99|1778.3999999999999|
|Thur|  62|  2.77|           1096.33|
| Fri|  19|  2.73|            325.88|
+----+----+------+------------------+



### 문제 6-2: 다중 컬럼 그룹 집계

Tips 데이터를 `day`와 `time`으로 그룹화하여 건수와 평균 팁을 계산하세요.

<details>
<summary>힌트</summary>

`groupBy("컬럼1", "컬럼2")`로 여러 컬럼 그룹화가 가능합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.groupBy("day", "time").agg(
    count("*").alias("count"),
    spark_round(avg("tip"), 2).alias("avg_tip")
).orderBy("day", "time").show()
```

</details>

In [ ]:
# [문제 6-2] 여기에 코드를 작성하세요
df_tips.groupBy("day", "time").agg(
    count("*").alias("건수"),
    spark_round(avg("tip"), 2).alias("평균팁"),
).show()


+----+------+----+------+
| day|  time|건수|평균팁|
+----+------+----+------+
| Sun|Dinner|  76|  3.26|
| Sat|Dinner|  87|  2.99|
|Thur| Lunch|  61|  2.77|
| Fri|Dinner|  12|  2.94|
| Fri| Lunch|   7|  2.38|
|Thur|Dinner|   1|   3.0|
+----+------+----+------+



### 문제 6-3: 조건부 집계

Tips 데이터에서 `day`별로 다음을 계산하세요:
- 전체 건수
- 흡연자(`smoker` == "Yes") 건수
- 비흡연자 건수
- 흡연자 비율 (%)

<details>
<summary>힌트</summary>

`count(when(조건, 1))`으로 조건에 맞는 행만 카운트합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.groupBy("day").agg(
    count("*").alias("total"),
    count(when(col("smoker") == "Yes", 1)).alias("smoker_cnt"),
    count(when(col("smoker") == "No", 1)).alias("non_smoker_cnt"),
    spark_round(
        count(when(col("smoker") == "Yes", 1)) / count("*") * 100, 1
    ).alias("smoker_rate")
).orderBy("day").show()
```

</details>

In [27]:
# [문제 6-3] 여기에 코드를 작성하세요

df_tips.groupBy("day").agg(
    count("*").alias("전체건수"),
    count(when(col("smoker") == "Yes",1)).alias("흡연자수"),
    count(when(col("smoker") == "No",1)).alias("비흡연자수"),
    spark_round(
        count(when(col("smoker") == "Yes",1)) / count("*")* 100, 1
        ).alias("흡연자비율")
).show()

+----+--------+--------+----------+----------+
| day|전체건수|흡연자수|비흡연자수|흡연자비율|
+----+--------+--------+----------+----------+
| Sun|      76|      19|        57|      25.0|
| Sat|      87|      42|        45|      48.3|
|Thur|      62|      17|        45|      27.4|
| Fri|      19|      15|         4|      78.9|
+----+--------+--------+----------+----------+



### 문제 6-4: Titanic 생존 분석

Titanic 데이터에서 `pclass`(객실 등급)별로 다음을 계산하세요:
- 총 승객 수
- 생존자 수
- 사망자 수
- 생존율 (%)

<details>
<summary>힌트</summary>

`survived` 컬럼은 0(사망) 또는 1(생존)입니다.
`sum("survived")`는 생존자 수가 됩니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_titanic.groupBy("pclass").agg(
    count("*").alias("total"),
    sum("survived").alias("survived"),
    count(when(col("survived") == 0, 1)).alias("died"),
    spark_round(avg("survived") * 100, 1).alias("survival_rate")
).orderBy("pclass").show()
```

</details>

In [29]:
df_titanic.show(10)

+--------+------+------+----+-----+-----+-------+--------+------+-----+----------+----+-----------+-----+-----+
|survived|pclass|   sex| age|sibsp|parch|   fare|embarked| class|  who|adult_male|deck|embark_town|alive|alone|
+--------+------+------+----+-----+-----+-------+--------+------+-----+----------+----+-----------+-----+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S| Third|  man|      true| NaN|Southampton|   no|false|
|       1|     1|female|38.0|    1|    0|71.2833|       C| First|woman|     false|   C|  Cherbourg|  yes|false|
|       1|     3|female|26.0|    0|    0|  7.925|       S| Third|woman|     false| NaN|Southampton|  yes| true|
|       1|     1|female|35.0|    1|    0|   53.1|       S| First|woman|     false|   C|Southampton|  yes|false|
|       0|     3|  male|35.0|    0|    0|   8.05|       S| Third|  man|      true| NaN|Southampton|   no| true|
|       0|     3|  male| NaN|    0|    0| 8.4583|       Q| Third|  man|      true| NaN| Queenstown|   no

In [32]:
# [문제 6-4] 여기에 코드를 작성하세요
df_titanic.groupBy("pclass").agg(
    count("*").alias("총승객수"),
    count(when(col("alive") == "yes",1)).alias("생존자수"),
    count(when(col("alive") == "no",1)).alias("사망자수"),
    spark_round(
        count(when(col("alive") == "yes",1)) / count("*")* 100, 1
        ).alias("생존자비율")
).show()


+------+--------+--------+--------+----------+
|pclass|총승객수|생존자수|사망자수|생존자비율|
+------+--------+--------+--------+----------+
|     3|     491|     119|     372|      24.2|
|     2|     184|      87|      97|      47.3|
|     1|     216|     136|      80|      63.0|
+------+--------+--------+--------+----------+



---

## Part 7: 피벗 테이블

### 문제 7-1: 기본 피벗

Tips 데이터로 `day`(행) × `time`(열) 형태의 피벗 테이블을 만들고,
각 셀에는 건수를 표시하세요.

<details>
<summary>힌트</summary>

`groupBy("행컬럼").pivot("열컬럼").count()`

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.groupBy("day").pivot("time").count().show()
```

</details>

In [12]:
df_tips.groupBy("day").pivot("time").count().show()

+----+------+-----+
| day|Dinner|Lunch|
+----+------+-----+
| Sun|    76| NULL|
| Fri|    12|    7|
|Thur|     1|   61|
| Sat|    87| NULL|
+----+------+-----+



In [ ]:
# [문제 7-1] 여기에 코드를 작성하세요
df_tips.groupBy("day").pivot("time").count().show()

+----+------+-----+
| day|Dinner|Lunch|
+----+------+-----+
| Sun|    76| NULL|
| Fri|    12|    7|
|Thur|     1|   61|
| Sat|    87| NULL|
+----+------+-----+



### 문제 7-2: 집계와 함께 피벗

Tips 데이터로 `day`(행) × `smoker`(열) 형태의 피벗 테이블을 만들고,
각 셀에는 평균 팁을 표시하세요. 소수점 2자리로 반올림하세요.

<details>
<summary>힌트</summary>

`pivot()` 후 `agg(spark_round(avg("컬럼"), 2))`를 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_tips.groupBy("day").pivot("smoker").agg(
    spark_round(avg("tip"), 2)
).show()
```

</details>

In [ ]:
# [문제 7-2] 여기에 코드를 작성하세요
df_tips.groupBy("day").pivot("smoker").agg(
            spark_round(avg("tip"),2)
).show()
    


+----+----+----+
| day|  No| Yes|
+----+----+----+
| Sun|3.17|3.52|
| Fri|2.81|2.71|
|Thur|2.67|3.03|
| Sat| 3.1|2.88|
+----+----+----+



### 문제 7-3: Titanic 피벗 분석

Titanic 데이터로 `pclass`(행) × `sex`(열) 형태의 피벗 테이블을 만들고,
각 셀에는 생존율(%)을 표시하세요.

<details>
<summary>힌트</summary>

`survived`의 평균 × 100이 생존율입니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_titanic.groupBy("pclass").pivot("sex").agg(
    spark_round(avg("survived") * 100, 1)
).show()
```

</details>

In [15]:
# [문제 7-3] 여기에 코드를 작성하세요
df_titanic.groupBy("pclass").pivot("sex").agg(
            spark_round(avg("survived") *100,1).alias("생존율")
).show()
    


+------+------+----+
|pclass|female|male|
+------+------+----+
|     3|  50.0|13.5|
|     2|  92.1|15.7|
|     1|  96.8|36.9|
+------+------+----+



---

## Part 8: 조인

### 데이터 준비

조인 실습을 위해 Tips 데이터에서 파생 테이블을 만듭니다.

In [16]:
# -----------------------------------------------------------------------------
# 조인용 데이터 준비
# -----------------------------------------------------------------------------

# 요일별 평균 정보 테이블 생성
df_day_stats = df_tips.groupBy("day").agg(
    spark_round(avg("total_bill"), 2).alias("avg_bill"),
    spark_round(avg("tip"), 2).alias("avg_tip"),
    count("*").alias("visit_count")
)

print("=== 요일별 통계 테이블 ===")
df_day_stats.show()

# 시간대별 정보 테이블 생성
df_time_stats = df_tips.groupBy("time").agg(
    spark_round(avg("total_bill"), 2).alias("avg_bill"),
    count("*").alias("count")
)

print("=== 시간대별 통계 테이블 ===")
df_time_stats.show()

=== 요일별 통계 테이블 ===
+----+--------+-------+-----------+
| day|avg_bill|avg_tip|visit_count|
+----+--------+-------+-----------+
| Sun|   21.41|   3.26|         76|
| Sat|   20.44|   2.99|         87|
|Thur|   17.68|   2.77|         62|
| Fri|   17.15|   2.73|         19|
+----+--------+-------+-----------+

=== 시간대별 통계 테이블 ===
+------+--------+-----+
|  time|avg_bill|count|
+------+--------+-----+
|Dinner|    20.8|  176|
| Lunch|   17.17|   68|
+------+--------+-----+



### 문제 8-1: Inner Join

Tips 데이터와 `df_day_stats`를 `day` 컬럼으로 Inner Join하세요.
결과에서 `total_bill`, `tip`, `day`, `avg_bill`, `avg_tip`을 선택하세요.

<details>
<summary>힌트</summary>

`df1.join(df2, "공통컬럼", "inner")`
같은 컬럼명으로 조인하면 문자열로 지정할 수 있습니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_joined = df_tips.join(df_day_stats, "day", "inner")
df_joined.select("total_bill", "tip", "day", "avg_bill", "avg_tip").show(10)
```

</details>

In [21]:
# [문제 8-1] 여기에 코드를 작성하세요
df_tips_join = df_tips.join(
    df_day_stats,
    "day",
    "inner"
)

df_tips_join.select(
    "total_bill","tip","day","avg_bill","avg_tip"
).show(5)


+----------+---+---+--------+-------+
|total_bill|tip|day|avg_bill|avg_tip|
+----------+---+---+--------+-------+
|     15.69|1.5|Sun|   21.41|   3.26|
|      23.1|4.0|Sun|   21.41|   3.26|
|     18.15|3.5|Sun|   21.41|   3.26|
|     30.46|2.0|Sun|   21.41|   3.26|
|      20.9|3.5|Sun|   21.41|   3.26|
+----------+---+---+--------+-------+
only showing top 5 rows



### 문제 8-2: 조인 후 파생 컬럼 추가

문제 8-1의 결과에서 `tip_vs_avg` 컬럼을 추가하세요.
이 컬럼은 `tip - avg_tip` (개인 팁 - 평균 팁)을 계산합니다.

<details>
<summary>힌트</summary>

조인 결과에 `withColumn()`을 체이닝합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_with_diff = df_tips.join(df_day_stats, "day", "inner").withColumn(
    "tip_vs_avg",
    spark_round(col("tip") - col("avg_tip"), 2)
)
df_with_diff.select("day", "tip", "avg_tip", "tip_vs_avg").show(10)
```

</details>

In [23]:
# [문제 8-2] 여기에 코드를 작성하세요
df_tips_join.withColumn(
    "tip_vs_avg",
    col("tip") / col("avg_tip")
).show(5)


+---+----------+---+------+------+------+----+--------+-------+-----------+------------------+
|day|total_bill|tip|   sex|smoker|  time|size|avg_bill|avg_tip|visit_count|        tip_vs_avg|
+---+----------+---+------+------+------+----+--------+-------+-----------+------------------+
|Sun|     15.69|1.5|  Male|   Yes|Dinner|   2|   21.41|   3.26|         76|0.4601226993865031|
|Sun|      23.1|4.0|  Male|   Yes|Dinner|   3|   21.41|   3.26|         76|1.2269938650306749|
|Sun|     18.15|3.5|Female|   Yes|Dinner|   3|   21.41|   3.26|         76|1.0736196319018405|
|Sun|     30.46|2.0|  Male|   Yes|Dinner|   5|   21.41|   3.26|         76|0.6134969325153374|
|Sun|      20.9|3.5|Female|   Yes|Dinner|   3|   21.41|   3.26|         76|1.0736196319018405|
+---+----------+---+------+------+------+----+--------+-------+-----------+------------------+
only showing top 5 rows



### 문제 8-3: Left Join과 NULL 확인

아래 코드로 새로운 요일 데이터를 만들고, `df_day_stats`와 Left Join하세요.
매칭되지 않는 행이 있는지 확인하세요.

```python
# 새로운 요일 포함 데이터
df_new_days = spark.createDataFrame([
    ("Mon", 25.0),
    ("Sat", 30.0),
    ("Wed", 20.0),
], ["day", "bill"])
```

<details>
<summary>힌트</summary>

Left Join 후 `avg_bill`이 NULL인 행을 필터링하면 매칭되지 않는 행을 찾을 수 있습니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_new_days = spark.createDataFrame([
    ("Mon", 25.0),
    ("Sat", 30.0),
    ("Wed", 20.0),
], ["day", "bill"])

df_left = df_new_days.join(df_day_stats, "day", "left")
df_left.show()

# 매칭되지 않는 행 확인
print("매칭되지 않는 요일:")
df_left.filter(col("avg_bill").isNull()).show()
```

</details>

In [ ]:
# [문제 8-3] 여기에 코드를 작성하세요
df_new_days = spark.createDataFrame([
    ("Mon", 25.0),
    ("Sat", 30.0),
    ("Wed", 20.0),
], ["day", "bill"])
# df_new_days를 기준으로 해서,df_day_stats에 day가 같은 행이 있으면 그 통계를 붙여서 보여주고, 없으면 df_new_days의 행은 그대로 두고 통계 컬럼은 NULL로 둔다.”
df_left =df_new_days.join(
    df_day_stats,
    "day",
    "left"
)
df_left.show()

df_left.filter(col("avg_bill").isNull()).show()


+---+----+--------+-------+-----------+
|day|bill|avg_bill|avg_tip|visit_count|
+---+----+--------+-------+-----------+
|Mon|25.0|    NULL|   NULL|       NULL|
|Sat|30.0|   20.44|   2.99|         87|
|Wed|20.0|    NULL|   NULL|       NULL|
+---+----+--------+-------+-----------+

+---+----+--------+-------+-----------+
|day|bill|avg_bill|avg_tip|visit_count|
+---+----+--------+-------+-----------+
|Mon|25.0|    NULL|   NULL|       NULL|
|Wed|20.0|    NULL|   NULL|       NULL|
+---+----+--------+-------+-----------+



---

## Part 9: NULL 처리

### 데이터 준비

NULL이 포함된 데이터를 생성합니다.

In [31]:
# -----------------------------------------------------------------------------
# NULL 처리용 데이터 준비
# -----------------------------------------------------------------------------

# Titanic 데이터의 NULL 확인
print("=== Titanic 데이터 NULL 현황 ===")
df_titanic.agg(
    count(when(col("age").isNull(), 1)).alias("age_null"),
    count(when(col("deck").isNull(), 1)).alias("deck_null"),
    count(when(col("embarked").isNull(), 1)).alias("embarked_null"),
    count(when(col("embark_town").isNull(), 1)).alias("embark_town_null"),
).show()

=== Titanic 데이터 NULL 현황 ===
+--------+---------+-------------+----------------+
|age_null|deck_null|embarked_null|embark_town_null|
+--------+---------+-------------+----------------+
|       0|        0|            0|               0|
+--------+---------+-------------+----------------+



### 문제 9-1: NULL 행 제거

Titanic 데이터에서 `age`가 NULL인 행을 제거하세요.
제거 전후의 행 수를 비교하세요.

<details>
<summary>힌트</summary>

`dropna(subset=["컬럼"])`을 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
print(f"원본 행 수: {df_titanic.count()}")

df_no_null_age = df_titanic.dropna(subset=["age"])
print(f"age NULL 제거 후: {df_no_null_age.count()}")
```

</details>

In [34]:
# [문제 9-1] 여기에 코드를 작성하세요
print(df_titanic.count())
df_no_age_null = df_titanic.dropna(subset=["age"])
df_no_age_null.count()

891


714

### 문제 9-2: NULL 값 채우기

Titanic 데이터에서:
- `age`의 NULL을 0으로 채우기
- `deck`의 NULL을 "Unknown"으로 채우기

<details>
<summary>힌트</summary>

`fillna({"컬럼1": 값1, "컬럼2": 값2})`로 컬럼별 다른 값을 지정합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_filled = df_titanic.fillna({
    "age": 0,
    "deck": "Unknown"
})

# NULL이 채워졌는지 확인
df_filled.agg(
    count(when(col("age").isNull(), 1)).alias("age_null"),
    count(when(col("deck").isNull(), 1)).alias("deck_null"),
).show()

# deck이 Unknown인 행 확인
df_filled.filter(col("deck") == "Unknown").select("survived", "pclass", "deck").show(5)
```

</details>

In [35]:
# [문제 9-2] 여기에 코드를 작성하세요
df_filled = df_titanic.fillna({
    "age": 0,                    
    "deck": "Unknown"     
})
df_filled.agg(
    count(when(col("age").isNull(),1)).alias("age_null"),
    count(when(col("deck").isNull(),1)).alias("deck_null")
).show()


+--------+---------+
|age_null|deck_null|
+--------+---------+
|       0|        0|
+--------+---------+



### 문제 9-3: coalesce로 대체값 설정

Titanic 데이터에서 `embarked` 컬럼의 NULL을 다음 우선순위로 채우세요:
1. `embarked` 값이 있으면 그대로
2. NULL이면 "S" (Southampton, 가장 많은 승선지)

<details>
<summary>힌트</summary>

`coalesce(col("컬럼"), lit("기본값"))`을 사용합니다.

</details>

<details>
<summary>모범 답안</summary>

```python
df_coalesced = df_titanic.withColumn(
    "embarked_filled",
    coalesce(col("embarked"), lit("S"))
)

# 확인
df_coalesced.filter(col("embarked").isNull()) \
    .select("embarked", "embarked_filled").show()
```

</details>

In [42]:
# [문제 9-3] 여기에 코드를 작성하세요
test = df_titanic.filter(col("embarked").isNull()).count()
print(test)

df_coalesced = df_titanic.withColumn(
    "embarked_filled",
    coalesce(
        col("embarked"),
        lit("S")
    )
)

df_coalesced.filter(col("embarked").isNull()) \
    .select("embarked", "embarked_filled").show()


0
+--------+---------------+
|embarked|embarked_filled|
+--------+---------------+
+--------+---------------+



---

## Part 10: Window 함수

### 문제 10-1: 그룹 내 순위 (row_number)

Tips 데이터에서 `day`별로 `total_bill` 내림차순 순위를 부여하세요.
컬럼명은 `bill_rank`로 하세요.

<details>
<summary>힌트</summary>

1. `Window.partitionBy("그룹컬럼").orderBy(col("정렬컬럼").desc())`로 윈도우 정의
2. `row_number().over(윈도우)`로 순위 부여

</details>

<details>
<summary>모범 답안</summary>

```python
window_day = Window.partitionBy("day").orderBy(col("total_bill").desc())

df_ranked = df_tips.withColumn(
    "bill_rank",
    row_number().over(window_day)
)

df_ranked.select("day", "total_bill", "tip", "bill_rank") \
    .orderBy("day", "bill_rank").show(20)
```

</details>

In [ ]:
# [문제 10-1] 여기에 코드를 작성하세요
window_rank = Window.partitionBy("day").orderBy(col("total_bill").desc())
#total_bill 값이 NULL이 아닌 행만 남긴 뒤,그 행들에 대해서만 window_rank 기준으로 순위를 매긴다.
df_ranked = df_tips.filter(col("total_bill").isNotNull()).withColumn(
    "bill_rank", row_number().over(window_rank))

df_ranked.show(5)

+----------+----+------+------+---+------+----+---------+
|total_bill| tip|   sex|smoker|day|  time|size|bill_rank|
+----------+----+------+------+---+------+----+---------+
|     40.17|4.73|  Male|   Yes|Fri|Dinner|   4|        1|
|     28.97| 3.0|  Male|   Yes|Fri|Dinner|   2|        2|
|     27.28| 4.0|  Male|   Yes|Fri|Dinner|   2|        3|
|     22.75|3.25|Female|    No|Fri|Dinner|   2|        4|
|     22.49| 3.5|  Male|    No|Fri|Dinner|   2|        5|
+----------+----+------+------+---+------+----+---------+
only showing top 5 rows



### 문제 10-2: 그룹별 Top N 추출

문제 10-1의 결과를 사용하여 각 요일별 `total_bill` Top 3를 추출하세요.

<details>
<summary>힌트</summary>

순위를 부여한 후 `filter(col("bill_rank") <= 3)`

</details>

<details>
<summary>모범 답안</summary>

```python
window_day = Window.partitionBy("day").orderBy(col("total_bill").desc())

df_top3 = df_tips.withColumn(
    "bill_rank", row_number().over(window_day)
).filter(col("bill_rank") <= 3)

df_top3.select("day", "total_bill", "tip", "bill_rank") \
    .orderBy("day", "bill_rank").show()
```

</details>

In [50]:
# [문제 10-2] 여기에 코드를 작성하세요
window_rank = Window.partitionBy("day").orderBy(col("total_bill").desc())

df_top3 = df_tips.filter(col("total_bill").isNotNull()).withColumn(
    "bill_rank", row_number().over(window_rank)
).filter(
    col("bill_rank") <= 3
)
df_top3.select("day", "total_bill", "tip", "bill_rank") \
    .orderBy("day", "bill_rank").show()


+----+----------+----+---------+
| day|total_bill| tip|bill_rank|
+----+----------+----+---------+
| Fri|     40.17|4.73|        1|
| Fri|     28.97| 3.0|        2|
| Fri|     27.28| 4.0|        3|
| Sat|     50.81|10.0|        1|
| Sat|     48.33| 9.0|        2|
| Sat|     48.27|6.73|        3|
| Sun|     48.17| 5.0|        1|
| Sun|     45.35| 3.5|        2|
| Sun|     40.55| 3.0|        3|
|Thur|     43.11| 5.0|        1|
|Thur|     41.19| 5.0|        2|
|Thur|     34.83|5.17|        3|
+----+----------+----+---------+



### 문제 10-3: 그룹 내 비율 계산

Tips 데이터에서 `day`별로 각 건의 `total_bill`이 그 날 전체 매출에서 차지하는 비율(%)을 계산하세요.
컬럼명은 `bill_pct`로 하세요.

<details>
<summary>힌트</summary>

1. `Window.partitionBy("day")`로 그룹 윈도우 정의 (정렬 없음)
2. `sum("total_bill").over(윈도우)`로 그룹 합계
3. 개별 값 / 그룹 합계 * 100

</details>

<details>
<summary>모범 답안</summary>

```python
window_day = Window.partitionBy("day")

df_pct = df_tips.withColumn(
    "day_total", sum("total_bill").over(window_day)
).withColumn(
    "bill_pct", spark_round(col("total_bill") / col("day_total") * 100, 2)
)

df_pct.select("day", "total_bill", "day_total", "bill_pct") \
    .orderBy("day", col("bill_pct").desc()).show(15)
```

</details>

In [ ]:
# [문제 10-3] 여기에 코드를 작성하세요



### 문제 10-4: lag/lead - 이전/다음 행 비교

Tips 데이터를 `total_bill` 순으로 정렬하고,
이전 행의 `total_bill`과 현재 행의 차이(`bill_diff`)를 계산하세요.

<details>
<summary>힌트</summary>

1. `Window.orderBy("total_bill")`로 전체 정렬 윈도우 정의
2. `lag("컬럼", 1).over(윈도우)`로 이전 행 값
3. 현재 값 - 이전 값

</details>

<details>
<summary>모범 답안</summary>

```python
window_order = Window.orderBy("total_bill")

df_with_lag = df_tips.withColumn(
    "prev_bill", lag("total_bill", 1).over(window_order)
).withColumn(
    "bill_diff", spark_round(col("total_bill") - col("prev_bill"), 2)
)

df_with_lag.select("total_bill", "prev_bill", "bill_diff").show(15)
```

</details>

In [ ]:
# [문제 10-4] 여기에 코드를 작성하세요



---

## Part 11: 종합 문제

### 종합 문제 1: Tips 데이터 분석 파이프라인

Tips 데이터를 사용하여 다음을 한 번에 수행하는 파이프라인을 작성하세요:

1. `tip_rate` 컬럼 추가 (tip / total_bill, 소수점 2자리)
2. `tip_rate`가 0.2(20%) 이상이면 "good_tipper"를, 아니면 "normal"을 `tipper_type` 컬럼에 저장
3. `day`와 `tipper_type`으로 그룹화하여 건수 집계
4. `day` 오름차순, 건수 내림차순 정렬

<details>
<summary>힌트</summary>

`withColumn().withColumn().groupBy().agg().orderBy()` 체이닝

</details>

<details>
<summary>모범 답안</summary>

```python
result = (
    df_tips
    .withColumn("tip_rate", spark_round(col("tip") / col("total_bill"), 2))
    .withColumn(
        "tipper_type",
        when(col("tip_rate") >= 0.2, "good_tipper").otherwise("normal")
    )
    .groupBy("day", "tipper_type")
    .agg(count("*").alias("count"))
    .orderBy(col("day").asc(), col("count").desc())
)

result.show()
```

</details>

In [ ]:
# [종합 문제 1] 여기에 코드를 작성하세요



### 종합 문제 2: Titanic 생존 분석

Titanic 데이터를 사용하여 다음을 수행하세요:

1. `age`의 NULL을 평균 나이로 채우기
2. `age_group` 컬럼 추가:
   - 12세 미만: "child"
   - 12~60세: "adult"
   - 60세 이상: "senior"
3. `pclass`와 `age_group`으로 피벗 테이블 생성 (값: 생존율 %)

<details>
<summary>힌트</summary>

1. 먼저 평균 나이를 계산 (`df.agg(avg("age")).collect()[0][0]`)
2. `fillna()`로 채우기
3. `when().when().otherwise()`로 분류
4. `pivot()`으로 피벗

</details>

<details>
<summary>모범 답안</summary>

```python
# 1. 평균 나이 계산
avg_age = df_titanic.agg(avg("age")).collect()[0][0]
print(f"평균 나이: {avg_age:.1f}")

# 2. NULL 채우기 + age_group 추가
df_processed = (
    df_titanic
    .fillna({"age": avg_age})
    .withColumn(
        "age_group",
        when(col("age") < 12, "child")
        .when(col("age") < 60, "adult")
        .otherwise("senior")
    )
)

# 3. 피벗 테이블 (생존율 %)
pivot_result = df_processed.groupBy("pclass").pivot("age_group", ["child", "adult", "senior"]).agg(
    spark_round(avg("survived") * 100, 1)
)

print("=== 객실 등급 × 연령대별 생존율 (%) ===")
pivot_result.orderBy("pclass").show()
```

</details>

In [ ]:
# [종합 문제 2] 여기에 코드를 작성하세요



### 종합 문제 3: Window 함수 종합

Titanic 데이터를 사용하여 다음을 수행하세요:

1. `pclass`별로 `fare` 내림차순 순위 부여 (`fare_rank`)
2. `pclass`별 `fare` 합계 대비 개인 비율(%) 계산 (`fare_pct`)
3. 각 등급별 Top 3 승객만 추출
4. 결과에서 `pclass`, `name`, `fare`, `fare_rank`, `fare_pct` 출력

<details>
<summary>힌트</summary>

- 순위용 Window: `partitionBy().orderBy()`
- 합계용 Window: `partitionBy()` (정렬 없음)
- Top 3: `filter(col("fare_rank") <= 3)`

</details>

<details>
<summary>모범 답안</summary>

```python
# Window 정의
window_rank = Window.partitionBy("pclass").orderBy(col("fare").desc())
window_sum = Window.partitionBy("pclass")

# 분석
df_fare_analysis = (
    df_titanic
    .withColumn("fare_rank", row_number().over(window_rank))
    .withColumn("class_fare_sum", sum("fare").over(window_sum))
    .withColumn("fare_pct", spark_round(col("fare") / col("class_fare_sum") * 100, 2))
    .filter(col("fare_rank") <= 3)
    .select("pclass", "name", "fare", "fare_rank", "fare_pct")
    .orderBy("pclass", "fare_rank")
)

df_fare_analysis.show(truncate=False)
```

</details>

In [ ]:
# [종합 문제 3] 여기에 코드를 작성하세요



---

## 과제 완료

축하합니다! PySpark DataFrame API 종합 과제를 완료했습니다.

### 학습 내용 정리

| 주제 | 주요 함수/메서드 |
|------|------------------|
| 컬럼 선택/조작 | `select()`, `withColumn()`, `withColumnRenamed()`, `drop()` |
| 필터링 | `filter()`, `where()`, `isin()`, `startswith()` |
| 조건부 값 | `when()`, `otherwise()` |
| 정렬/제한 | `orderBy()`, `limit()`, `distinct()` |
| 집계 | `groupBy()`, `agg()`, `count()`, `sum()`, `avg()` |
| 피벗 | `pivot()` |
| 조인 | `join()` (inner, left, right, outer) |
| NULL 처리 | `dropna()`, `fillna()`, `coalesce()` |
| Window 함수 | `row_number()`, `rank()`, `lag()`, `lead()`, `sum().over()` |

In [ ]:
# 세션 종료
# spark.stop()
print("\n과제 완료!")

# 2교시: Spark Structured Streaming

![Spark Structured Streaming](https://moons08.github.io/assets/img/post/spark/streaming-arch.png)

---

## 수업 목표

이 교시를 마치면 다음을 할 수 있습니다:

- Spark Structured Streaming의 개념과 특징을 이해할 수 있다
- Kafka에서 실시간으로 데이터를 읽어올 수 있다
- 윈도우 기반 집계(시간 윈도우)를 구현할 수 있다
- 스트리밍 결과를 콘솔/파일/Kafka로 출력할 수 있다

---

## 핵심 개념 1: 배치 처리 vs 스트리밍 처리

### 두 가지 데이터 처리 방식

데이터 처리 방식은 크게 **배치(Batch)**와 **스트리밍(Streaming)**으로 나뉩니다.

![](https://k21academy.com/wp-content/uploads/2020/11/BatchProcessingStreamProcessing_Diagram-02.png)

### 배치 처리 (Batch Processing)

**흐름**: 어제의 데이터 → 처리 (한번에) → 결과 (다음날 확인)

**예시**:
- 매일 밤 12시에 전날 판매 리포트 생성
- 매주 월요일에 주간 사용자 통계 계산
- 매월 1일에 월간 정산 처리

**특징**:
- 데이터가 모두 준비된 후 처리
- 높은 처리량 (throughput) 가능
- 지연 시간(latency)이 김 (시간~일)

### 스트리밍 처리 (Stream Processing)

**흐름**: 실시간 데이터 → 처리 → 결과 → 처리 → 결과 (끊임없이 반복)

**예시**:
- 실시간 대시보드 (트래픽 모니터링)
- 이상 거래 탐지 (1초 내 알림)
- 실시간 추천 시스템

**특징**:
- 데이터가 도착하는 즉시 처리
- 낮은 지연 시간(latency) (밀리초~초)
- 24시간 계속 실행

### 비교 요약

| 구분 | 배치 처리 | 스트리밍 처리 |
|------|----------|--------------|
| 처리 시점 | 데이터 축적 후 일괄 처리 | 데이터 도착 즉시 처리 |
| 지연 시간 | 시간 ~ 일 | 밀리초 ~ 초 |
| 처리량 | 높음 | 상대적으로 낮음 |
| 실행 방식 | 스케줄 기반 (주기적) | 24시간 상시 실행 |
| 사용 사례 | 리포트, 정산, 분석 | 모니터링, 알림, 실시간 대시보드 |

### 언제 어떤 방식을 사용하나요?

| 상황 | 배치 | 스트리밍 |
|------|------|----------|
| 데이터 분석 리포트 | O | |
| 실시간 대시보드 | | O |
| 월간 정산 | O | |
| 이상 거래 탐지 | | O |
| 머신러닝 모델 학습 | O | |
| 실시간 추천 | | O |
| ETL 파이프라인 | O | O (둘 다 가능) |

## 핵심 개념 2: Micro-batch 처리 방식

### Spark Structured Streaming의 동작 원리

Spark Structured Streaming은 **Micro-batch** 방식으로 스트리밍을 처리합니다.
데이터를 작은 배치 단위로 나누어 처리하는 방식입니다.

**처리 흐름**:
1. 데이터 도착 → 배치 단위로 분할
2. 각 배치별 처리 실행
3. 결과 출력
4. 다음 배치 반복

예: `trigger: processingTime="5 seconds"` → 5초마다 배치 처리

### Micro-batch의 장점

| 장점 | 설명 |
|------|------|
| **Exactly-once 보장** | 데이터 중복/손실 없이 정확히 한 번 처리 |
| **배치 코드 재사용** | DataFrame API를 그대로 사용 |
| **장애 복구 용이** | 체크포인트로 상태 저장/복구 |
| **간편한 개발** | 스트리밍 복잡성을 추상화 |

### 트리거(Trigger) 옵션

| 트리거 | 설명 | 사용 예시 |
|--------|------|----------|
| `processingTime="5 seconds"` | 5초마다 배치 처리 | 실시간 대시보드 |
| `processingTime="1 minute"` | 1분마다 배치 처리 | 준실시간 집계 |
| `once=True` | 한 번만 처리 | 배치 스타일 스트리밍 |
| `availableNow=True` | 현재까지 데이터 한 번 처리 | 백필(backfill) |

### 리얼타임 모드(Real-Time Mode, RTM)

Spark 4.1부터 도입된 **초저지연 연속 처리** 모드입니다.
Micro-batch의 배치 간격으로 인한 지연을 제거하여 **p99 레이턴시를 한 자릿수 밀리초**까지 낮출 수 있습니다.

**Micro-batch vs Real-Time Mode**:

![Code Diff btw rtm simple change](https://www.databricks.com/sites/default/files/inline-images/image5_36.png)

| 구분 | Micro-batch | Real-Time Mode |
|------|-------------|----------------|
| **처리 방식** | 주기적 배치 처리 | 연속(Continuous) 처리 |
| **레이턴시** | 초~분 단위 | 밀리초 단위 |
| **적용 방식** | 기본 모드 | 설정 변경만으로 활성화 |

**RTM 지원 범위 (Spark 4.1 기준)**:

| 항목 | 지원 내용 |
|------|-----------|
| **언어** | Scala |
| **쿼리 타입** | Stateless / 단일 스테이지 |
| **Source** | Kafka |
| **Sink** | Kafka, ForeachSink |
| **출력 모드** | Update 모드만 |

> 💡 **선택 기준**: 대부분의 스트리밍 워크로드는 Micro-batch로 충분합니다.
> 밀리초 단위 응답이 필수인 경우에만 RTM을 고려하세요.

## 핵심 개념 3: Watermark (워터마크)

### 지연 데이터 문제

실시간 시스템에서는 네트워크 지연 등으로 데이터가 늦게 도착할 수 있습니다.
Watermark는 "얼마나 늦은 데이터까지 허용할 것인가"를 정의합니다.

**예시**: Watermark = 10분 설정

| 실제 시간 | 데이터 | 이벤트 시간 | 처리 여부 |
|-----------|--------|-------------|-----------|
| 10:00 | A | 10:00 | 처리 |
| 10:05 | B | 10:04 | 처리 |
| 10:10 | C | 10:08 | 처리 |
| 10:15 | D | 10:12 | 처리 |
| 10:20 | E | 10:03 (늦음!) | **버림** |

**계산**: 현재 시간 10:15일 때, Watermark = 10:15 - 10분 = 10:05
- 이벤트 시간 10:03인 (E)는 10:05보다 이전 → **버림**
- 결론: 10분 이상 늦은 데이터는 무시됨

### Watermark 설정 방법

```python
df.withWatermark("event_time", "10 minutes")
```

| 파라미터 | 의미 |
|----------|------|
| `"event_time"` | 이벤트 시간 컬럼명 |
| `"10 minutes"` | 허용할 최대 지연 시간 |

### Watermark 설정 기준

| 값 | 상황 |
|----|------|
| 작은 값 (1분) | 네트워크가 안정적, 빠른 결과 필요 |
| 큰 값 (1시간) | 네트워크가 불안정, 데이터 손실 최소화 |
| 중간 값 (10분) | 일반적인 상황 |

## 핵심 개념 4: 윈도우 집계 (Window Aggregation)

### 시간 윈도우란?

![](https://www.databricks.com/wp-content/uploads/2021/10/Native-Support-4-Session-Window-in-Spark-Streaming-blog-img-1.jpg)

스트리밍 데이터를 시간 단위로 묶어서 집계하는 방법입니다.

### Tumbling Window (텀블링 윈도우)

윈도우가 **겹치지 않음** - 각 데이터는 하나의 윈도우에만 속함

| 윈도우 | 시간 범위 | 설명 |
|--------|----------|------|
| Window 1 | 0~5분 | 집계 결과 1 |
| Window 2 | 5~10분 | 집계 결과 2 |
| Window 3 | 10~15분 | 집계 결과 3 |

**특징**: 가장 일반적으로 사용

### Sliding Window (슬라이딩 윈도우)

윈도우가 **겹침** - 하나의 데이터가 여러 윈도우에 속할 수 있음

| 윈도우 | 시간 범위 |
|--------|----------|
| Window 1 | 0~5분 |
| Window 2 | 2~7분 |
| Window 3 | 4~9분 |
| Window 4 | 6~11분 |

**특징**: 더 세밀한 분석 가능

### 윈도우 설정 방법

```python
# Tumbling Window (5분)
window(col("event_time"), "5 minutes")

# Sliding Window (5분 윈도우, 1분 슬라이드)
window(col("event_time"), "5 minutes", "1 minute")
```

## Structured Streaming이란?

### 개념

**배치 처리 (기존 방식)**: 데이터 전체 로드 → 처리 → 결과 출력

**스트리밍 처리 (Structured Streaming)**: 데이터 조금씩 → 처리 → 결과 지속 업데이트 (micro-batch 반복)

### 핵심 아이디어: 무한 테이블

스트리밍 데이터를 "무한히 늘어나는 테이블"처럼 취급합니다.

| 시점 | 테이블 상태 |
|------|------------|
| 초기 | row 1, row 2 |
| 새 데이터 도착 | row 1, row 2, **row 3 (new)**, **row 4 (new)** |
| 또 새 데이터 | row 1, row 2, row 3, row 4, **row 5 (new)** |

**장점**: 같은 DataFrame API로 배치/스트리밍 모두 처리 가능

---

# 3교시 Spark Structured Streaming 패턴 실습

---

## 수업 목표

이 교안을 마치면 다음을 할 수 있습니다:

- 다양한 스트리밍 소스(rate, socket, kafka, file)를 활용할 수 있다
- 스트림 데이터에 스키마를 정의하고 JSON을 파싱할 수 있다
- 스트림에 filter, select, withColumn 등 변환 연산을 적용할 수 있다
- Output Mode(append, update, complete)를 상황에 맞게 선택할 수 있다
- 다양한 Sink(console, file, kafka, foreachBatch)로 결과를 출력할 수 있다
- Trigger를 설정하여 처리 주기를 제어할 수 있다
- Tumbling/Sliding 윈도우 집계를 구현할 수 있다
- Watermark로 지연 데이터를 처리할 수 있다
- 체크포인트로 장애 복구를 구현할 수 있다

---

## 실습 환경 구성

### 필요한 파일

이 교안은 `spark-kafka-compose` 환경에서 실행합니다.
아래 파일들이 `/app` 디렉토리에 있어야 합니다.

#### 1. `producer.py` - Kafka 테스트 데이터 생성기

```python
"""
Kafka Producer - API 이벤트 생성기

실행 방법:
    docker exec -it python-dev python producer.py --rate 10
"""
import json
import time
import random
import uuid
import argparse
from datetime import datetime
from confluent_kafka import Producer

KAFKA_BOOTSTRAP_SERVERS = "kafka:9092"
TOPIC_NAME = "api-events"

ENDPOINTS = ["/api/users", "/api/orders", "/api/products", "/api/payments", "/api/search"]
METHODS = ["GET", "POST", "PUT", "DELETE"]
STATUS_CODES = [
    (200, 60), (201, 10), (400, 8), (401, 5),
    (404, 7), (500, 6), (502, 2), (503, 2),
]

def weighted_choice(choices):
    total = sum(weight for _, weight in choices)
    r = random.uniform(0, total)
    cumulative = 0
    for item, weight in choices:
        cumulative += weight
        if r <= cumulative:
            return item
    return choices[-1][0]

def generate_event():
    status_code = weighted_choice(STATUS_CODES)
    if status_code >= 500:
        response_time = random.randint(500, 2000)
    elif status_code >= 400:
        response_time = random.randint(100, 500)
    else:
        response_time = random.randint(10, 300)
    return {
        "request_id": str(uuid.uuid4())[:8],
        "user_id": f"user-{random.randint(1, 100):03d}",
        "endpoint": random.choice(ENDPOINTS),
        "method": random.choice(METHODS),
        "status_code": status_code,
        "response_time_ms": response_time,
        "timestamp": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
    }

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--rate", type=int, default=10)
    parser.add_argument("--duration", type=int, default=0)
    args = parser.parse_args()

    producer = Producer({"bootstrap.servers": KAFKA_BOOTSTRAP_SERVERS})
    interval = 1.0 / args.rate
    total_sent = 0
    start_time = time.time()

    try:
        while True:
            if args.duration > 0 and (time.time() - start_time) >= args.duration:
                break
            event = generate_event()
            producer.produce(TOPIC_NAME, key=event["endpoint"], value=json.dumps(event))
            total_sent += 1
            if total_sent % 10 == 0:
                print(f"전송: {total_sent}건")
            producer.poll(0)
            time.sleep(interval)
    except KeyboardInterrupt:
        pass
    finally:
        producer.flush()
        print(f"총 전송: {total_sent}건")

if __name__ == "__main__":
    main()
```

---

### 환경 시작

```bash
cd docker
docker compose up -d
```

### 실습 진행 방법

**터미널 1: Spark 실습 코드 실행**
```bash
docker exec -it python-dev bash
cd /app
python streaming_lesson.py
```

**터미널 2: Kafka Producer 실행 (Kafka 예제 시)**
```bash
docker exec -it python-dev python producer.py --rate 10
```

> **핵심**: 각 셀을 실행하면 **Console에 결과가 출력**됩니다!
> 스트리밍 쿼리를 멈추려면 `Ctrl+C`를 누르세요.

---

## Part 0: 첫 스트리밍 실행 - 바로 결과 보기

복잡한 설정 없이 **바로 스트리밍 결과를 확인**해봅시다.
Rate 소스는 테스트용 더미 데이터를 자동 생성하므로 외부 의존성이 없습니다.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, when, count, sum as spark_sum, avg, min, max,
    from_json, to_json, struct, explode, split,
    window, current_timestamp, expr, to_timestamp, round as spark_round
)
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType,
    LongType, DoubleType, TimestampType, ArrayType, BooleanType
)
import time

# SparkSession 생성 (로컬 모드)
spark = (
    SparkSession.builder
    .appName("Structured-Streaming-Lesson")
    .master("local[*]")
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.8")
    .config("spark.sql.shuffle.partitions", 4)
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
print("SparkSession 생성 완료!")

:: loading settings :: url = jar:file:/usr/local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c363a27f-7fb2-494d-90e1-c6299a08268f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.8 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.8 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-

SparkSession 생성 완료!


26/01/21 07:39:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### 0.1 Rate 소스 원본 데이터 확인

먼저 Rate 소스가 **어떤 데이터를 생성하는지** 직접 확인해봅시다.
변환 없이 원본 그대로 출력합니다.

In [2]:
# =============================================================================
# Rate 소스 원본 데이터 확인
# =============================================================================
print("=" * 60)
print("Rate 소스 원본 데이터")
print("- Rate 소스가 자동 생성하는 데이터를 확인합니다")
print("- timestamp: 생성 시간")
print("- value: 0부터 시작하는 순차 번호")
print("=" * 60)

df_rate_raw = (
    spark.readStream
    .format("rate")
    .option("rowsPerSecond", 3)  # 초당 3개 생성
    .load()
)

# 원본 그대로 출력
query = (
    df_rate_raw.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(10)
query.stop()
print("\n원본 데이터 확인 완료!")

Rate 소스 원본 데이터
- Rate 소스가 자동 생성하는 데이터를 확인합니다
- timestamp: 생성 시간
- value: 0부터 시작하는 순차 번호


26/01/21 07:41:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7dddbfc4-f8d6-4bd4-bc4d-1cd39e531311. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
26/01/21 07:41:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2026-01-21 07:41:54.415|0    |
|2026-01-21 07:41:54.748|1    |
|2026-01-21 07:41:55.082|2    |
|2026-01-21 07:41:55.415|3    |
|2026-01-21 07:41:55.748|4    |
|2026-01-21 07:41:56.082|5    |
+-----------------------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2026-01-21 07:41:56.415|6    |
|2026-01-21 07:41:56.748|7    |
|2026-01-21 07:41:57.082|8    |
|2026-01-21 07:41:57.415|9    |
|2026-01-21 07:41:57.748|10   |
|2026-01-21 07:41:58.082|11   |
|2026-01-21 07:41:58.415|12

**Rate 소스 원본 출력**:
```
-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-----+
|timestamp           |value|
+--------------------+-----+
|2026-01-21 10:00:00 |0    |
|2026-01-21 10:00:00 |1    |
|2026-01-21 10:00:00 |2    |
+--------------------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+
|timestamp           |value|
+--------------------+-----+
|2026-01-21 10:00:03 |3    |
|2026-01-21 10:00:03 |4    |
|2026-01-21 10:00:03 |5    |
|2026-01-21 10:00:04 |6    |
|2026-01-21 10:00:04 |7    |
|2026-01-21 10:00:04 |8    |
+--------------------+-----+
```

**핵심 포인트**:
- `timestamp`: Spark가 데이터를 생성한 시간
- `value`: 0부터 시작하여 **순차적으로 증가**하는 long 타입 숫자
- 각 배치에 새로운 데이터만 포함됨 (append 모드)
- Rate 소스는 **외부 의존성 없이** 스트리밍 테스트 가능!

### 0.2 Rate 소스에 변환 적용

이제 원본 데이터에 변환을 적용해봅시다.
**실행 후 콘솔 출력을 확인하세요!**

In [3]:
# =============================================================================
# Rate 소스에 변환 적용
# =============================================================================
print("=" * 60)
print("Rate 소스 + 변환")
print("- 원본 value에 짝수/홀수 분류 추가")
print("=" * 60)

df_rate = (
    spark.readStream
    .format("rate")
    .option("rowsPerSecond", 5)
    .load()
)

# 간단한 변환: 짝수/홀수 구분
df_transformed = (
    df_rate
    .withColumn("is_even", col("value") % 2 == 0)
    .withColumn("category",
        when(col("value") % 2 == 0, "짝수").otherwise("홀수")
    )
)

query = (
    df_transformed.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(12)
query.stop()
print("\n변환 적용 스트리밍 종료!")

Rate 소스 + 변환
- 원본 value에 짝수/홀수 분류 추가


26/01/21 07:44:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0e30e3c3-65f7-4b8b-9b40-0951ea9843e5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
26/01/21 07:44:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+-------+--------+
|timestamp|value|is_even|category|
+---------+-----+-------+--------+
+---------+-----+-------+--------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-----+-------+--------+
|timestamp              |value|is_even|category|
+-----------------------+-----+-------+--------+
|2026-01-21 07:44:19.216|0    |true   |짝수    |
|2026-01-21 07:44:19.416|1    |false  |홀수    |
|2026-01-21 07:44:19.616|2    |true   |짝수    |
|2026-01-21 07:44:19.816|3    |false  |홀수    |
|2026-01-21 07:44:20.016|4    |true   |짝수    |
+-----------------------+-----+-------+--------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------------+-----+-------+--------+
|timestamp              |value|is_even|category|
+-----------------------+-----+-------

**변환 적용 후 출력**:
```
-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+-------+--------+
|timestamp           |value|is_even|category|
+--------------------+-----+-------+--------+
|2026-01-21 10:00:00 |0    |true   |짝수    |
|2026-01-21 10:00:00 |1    |false  |홀수    |
|2026-01-21 10:00:00 |2    |true   |짝수    |
|2026-01-21 10:00:00 |3    |false  |홀수    |
+--------------------+-----+-------+--------+
```

---

## Part 1: 스트리밍 소스 패턴

Structured Streaming은 다양한 소스에서 데이터를 읽을 수 있습니다.

| 소스 | 용도 | 사용 상황 |
|------|------|----------|
| `rate` | 테스트용 더미 데이터 | 개발/디버깅 |
| `socket` | TCP 소켓 | 간단한 테스트 |
| `kafka` | Kafka 토픽 | 프로덕션 실시간 처리 |
| `file` | 파일 디렉토리 | 배치 스타일 스트리밍 |

### 1.1 Rate 소스 (테스트용)

초당 지정된 수의 행을 자동 생성합니다. **외부 의존성 없이** 스트리밍을 테스트할 수 있습니다.

In [4]:
# =============================================================================
# Rate 소스: 테스트용 더미 데이터 자동 생성
# =============================================================================
df_rate = (
    spark.readStream
    .format("rate")
    .option("rowsPerSecond", 10)   # 초당 10개
    .load()
)

print("Rate 소스 스키마:")
df_rate.printSchema()
# root
#  |-- timestamp: timestamp
#  |-- value: long

Rate 소스 스키마:
root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



**Rate 소스 옵션**:

| 옵션 | 설명 | 기본값 |
|------|------|--------|
| `rowsPerSecond` | 초당 생성 행 수 | 1 |
| `rampUpTime` | 목표 속도까지 도달 시간 | 0s |
| `numPartitions` | 파티션 수 | Spark 기본값 |

### 1.2 Socket 소스 (간단한 테스트)

TCP 소켓에서 텍스트 데이터를 읽습니다.
터미널에서 `nc -lk 9999` 실행 후 텍스트 입력하면 수신됩니다.

In [ ]:
# =============================================================================
# Socket 소스: TCP 소켓에서 텍스트 읽기 (참고용, 실행 시 nc 필요)
# =============================================================================
# df_socket = (
#     spark.readStream
#     .format("socket")
#     .option("host", "localhost")
#     .option("port", 9999)
#     .load()
# )
# 스키마: value (StringType) 하나의 컬럼
print("Socket 소스는 프로덕션에서 사용하지 않습니다 (장애 복구 불가)")

### 1.3 Kafka 소스 (프로덕션)

가장 많이 사용하는 프로덕션 소스입니다.

In [5]:
# =============================================================================
# Kafka 메시지 스키마
# =============================================================================
# Kafka에서 읽은 메시지는 다음 컬럼을 포함합니다:
# - key: binary
# - value: binary        ← 실제 데이터 (JSON 등)
# - topic: string
# - partition: integer
# - offset: long
# - timestamp: timestamp
# - timestampType: integer

# API 이벤트 JSON 스키마 정의
event_schema = StructType([
    StructField("request_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("endpoint", StringType(), True),
    StructField("method", StringType(), True),
    StructField("status_code", IntegerType(), True),
    StructField("response_time_ms", IntegerType(), True),
    StructField("timestamp", StringType(), True),
])

print("이벤트 스키마 정의 완료!")
print(event_schema)

이벤트 스키마 정의 완료!
StructType([StructField('request_id', StringType(), True), StructField('user_id', StringType(), True), StructField('endpoint', StringType(), True), StructField('method', StringType(), True), StructField('status_code', IntegerType(), True), StructField('response_time_ms', IntegerType(), True), StructField('timestamp', StringType(), True)])


**Kafka 소스 주요 옵션**:

| 옵션 | 설명 | 예시 |
|------|------|------|
| `subscribe` | 단일/여러 토픽 구독 | `"topic1,topic2"` |
| `subscribePattern` | 패턴으로 토픽 구독 | `"api-.*"` |
| `startingOffsets` | 시작 위치 | `"earliest"`, `"latest"` |
| `maxOffsetsPerTrigger` | 트리거당 최대 오프셋 | `10000` |
| `kafka.group.id` | Consumer Group ID | `"my-group"` |

### 1.4 File 소스 (배치 스타일)

디렉토리에 새로 추가되는 파일을 스트림으로 처리합니다.

In [ ]:
# =============================================================================
# File 소스: 새 파일 자동 감지 및 처리 (참고용)
# =============================================================================
file_schema = StructType([
    StructField("id", StringType(), True),
    StructField("value", IntegerType(), True),
    StructField("timestamp", StringType(), True),
])

# df_file = (
#     spark.readStream
#     .format("json")                      # 또는 "csv", "parquet"
#     .schema(file_schema)                 # 스키마 필수!
#     .option("maxFilesPerTrigger", 10)    # 트리거당 최대 파일 수
#     .load("/data/input/")                # 감시할 디렉토리
# )
print("File 소스는 스키마 지정이 필수입니다!")

---

## Part 2: 스키마 정의와 JSON 파싱

Kafka 메시지는 바이너리로 전송되므로 JSON 파싱이 필요합니다.

### 2.1 주요 데이터 타입

| 타입 | 설명 | 예시 |
|------|------|------|
| `StringType()` | 문자열 | `"hello"` |
| `IntegerType()` | 32비트 정수 | `123` |
| `LongType()` | 64비트 정수 | `123456789` |
| `DoubleType()` | 64비트 실수 | `3.14` |
| `BooleanType()` | 불리언 | `True` |
| `TimestampType()` | 타임스탬프 | `2024-01-01 00:00:00` |
| `ArrayType(elem)` | 배열 | `[1, 2, 3]` |
| `MapType(k, v)` | 맵 | `{"a": 1}` |

### 2.2 JSON 파싱 패턴

In [ ]:
# =============================================================================
# JSON 파싱 시뮬레이션 (Rate 소스로)
# =============================================================================
# 실제 Kafka 파싱 흐름:
# 1) col("value").cast("string"): 바이너리 → 문자열
# 2) from_json(문자열, 스키마): JSON 문자열 → 구조체
# 3) .alias("data"): 구조체에 별칭 부여
# 4) select("data.*"): 구조체 내부 필드를 개별 컬럼으로 펼침

print("=" * 60)
print("JSON 파싱 패턴 (Rate 소스로 시뮬레이션)")
print("=" * 60)

# Rate 소스로 JSON 파싱 시뮬레이션
df_rate = spark.readStream.format("rate").option("rowsPerSecond", 3).load()

# value를 JSON처럼 변환하여 파싱 연습
df_simulated = (
    df_rate
    .withColumn("json_str",
        struct(
            col("value").alias("id"),
            (col("value") * 10).alias("amount"),
            col("timestamp").alias("event_time")
        )
    )
    .select(
        col("json_str.id"),
        col("json_str.amount"),
        col("json_str.event_time")
    )
)

query = (
    df_simulated.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(10)
query.stop()
print("\nJSON 파싱 시뮬레이션 종료!")

### 2.3 중첩 JSON 파싱 (실습)

실제로 중첩된 JSON을 파싱해보겠습니다.
시뮬레이션 데이터로 **중첩 구조 접근 방법**을 익힙니다.

In [ ]:
# =============================================================================
# 중첩 JSON 파싱 실습
# =============================================================================
print("=" * 60)
print("중첩 JSON 파싱 실습")
print("- 중첩된 user 객체에서 필드 추출")
print("=" * 60)

# 중첩 JSON 스키마 정의
# JSON 예시: {"user": {"id": "U001", "name": "Alice"}, "event": "click", "amount": 100}
nested_schema = StructType([
    StructField("user", StructType([
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
    ]), True),
    StructField("event", StringType(), True),
    StructField("amount", IntegerType(), True),
])

# 테스트용 JSON 데이터 생성 (배치 DataFrame으로 시뮬레이션)
nested_json_data = [
    ('{"user": {"id": "U001", "name": "Alice"}, "event": "click", "amount": 100}',),
    ('{"user": {"id": "U002", "name": "Bob"}, "event": "purchase", "amount": 250}',),
    ('{"user": {"id": "U003", "name": "Charlie"}, "event": "view", "amount": 0}',),
]
df_nested_raw = spark.createDataFrame(nested_json_data, ["json_str"])

print("\n[1단계] 원본 JSON 문자열:")
df_nested_raw.show(truncate=False)

# JSON 파싱
df_nested_parsed = (
    df_nested_raw
    .select(from_json(col("json_str"), nested_schema).alias("data"))
)

print("[2단계] from_json 적용 후 (struct 타입):")
df_nested_parsed.show(truncate=False)
df_nested_parsed.printSchema()

# 중첩 필드 접근
df_nested_flat = (
    df_nested_parsed
    .select(
        col("data.user.id").alias("user_id"),       # 중첩 접근: data.user.id
        col("data.user.name").alias("user_name"),   # 중첩 접근: data.user.name
        col("data.event"),
        col("data.amount")
    )
)

print("[3단계] 중첩 필드를 개별 컬럼으로 추출:")
df_nested_flat.show(truncate=False)

**중첩 JSON 파싱 핵심**:
```python
# 1. 스키마 정의 - StructType 안에 StructType
nested_schema = StructType([
    StructField("user", StructType([        # 중첩 객체
        StructField("id", StringType()),
        StructField("name", StringType()),
    ])),
    StructField("event", StringType()),
])

# 2. 파싱 후 중첩 접근
df.select(
    col("data.user.id").alias("user_id"),   # 점(.) 표기법으로 접근
    col("data.user.name").alias("user_name"),
)
```

### 2.4 배열 타입 파싱 (실습)

배열이 포함된 JSON을 파싱하고, **explode로 행 단위로 펼치는** 방법을 익힙니다.

In [ ]:
# =============================================================================
# 배열 타입 JSON 파싱 실습
# =============================================================================
print("=" * 60)
print("배열 타입 JSON 파싱 실습")
print("- 배열 필드를 explode로 펼치기")
print("=" * 60)

# 배열 포함 JSON 스키마
# JSON 예시: {"order_id": "O001", "items": ["laptop", "mouse", "keyboard"]}
array_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("items", ArrayType(StringType()), True),
])

# 테스트용 JSON 데이터
array_json_data = [
    ('{"order_id": "O001", "items": ["laptop", "mouse", "keyboard"]}',),
    ('{"order_id": "O002", "items": ["phone", "charger"]}',),
    ('{"order_id": "O003", "items": ["tablet"]}',),
]
df_array_raw = spark.createDataFrame(array_json_data, ["json_str"])

print("\n[1단계] 원본 JSON 문자열:")
df_array_raw.show(truncate=False)

# JSON 파싱
df_array_parsed = (
    df_array_raw
    .select(from_json(col("json_str"), array_schema).alias("data"))
    .select("data.*")
)

print("[2단계] 파싱 후 (items는 배열 타입):")
df_array_parsed.show(truncate=False)
df_array_parsed.printSchema()

# 배열을 개별 행으로 펼치기
df_exploded = (
    df_array_parsed
    .select(
        col("order_id"),
        explode(col("items")).alias("item")  # 배열 → 개별 행
    )
)

print("[3단계] explode 적용 후 (각 아이템이 별도 행):")
df_exploded.show(truncate=False)

**배열 JSON 파싱 핵심**:
```python
# 1. 스키마 정의 - ArrayType 사용
array_schema = StructType([
    StructField("order_id", StringType()),
    StructField("items", ArrayType(StringType())),  # 문자열 배열
])

# 2. explode로 배열을 개별 행으로
df.select(
    col("order_id"),
    explode(col("items")).alias("item")  # ["a","b","c"] → 3개 행
)

# 참고: 빈 배열이면 해당 행 제거됨
# 빈 배열도 유지하려면 explode_outer() 사용
```

| 함수 | 빈 배열 처리 |
|------|-------------|
| `explode()` | 행 제거 |
| `explode_outer()` | NULL로 유지 |

---

## Part 3: 스트림 변환 패턴

배치 DataFrame에서 사용하던 변환 연산을 스트림에도 동일하게 적용할 수 있습니다.

### 3.1 지원되는 연산 vs 지원되지 않는 연산

| 지원됨 | 지원 안 됨 |
|--------|-----------|
| `select`, `filter`, `where` | `count()`, `collect()` (직접 호출) |
| `withColumn`, `drop` | `distinct()` (스트림 전체에 대해) |
| `groupBy` + 집계 | `sort()` (전체 정렬) |
| `join` (특정 조건) | `limit()`, `take()` |

> 💡 **핵심**: 무한 데이터를 전제로 하므로, "전체" 개념이 필요한 연산은 지원되지 않음

### 3.2 filter / where

In [ ]:
# =============================================================================
# filter / where: 조건에 맞는 행만 선택
# =============================================================================
print("=" * 60)
print("filter 변환 실습")
print("- value > 10 필터링")
print("- 짝수만 필터링")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# 여러 조건 AND
df_filtered = df_rate.filter(
    (col("value") > 10) &
    (col("value") % 2 == 0)  # 10보다 크고 짝수
)

query = (
    df_filtered.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(12)
query.stop()
print("\nfilter 실습 종료!")

### 3.3 select / withColumn / drop

In [ ]:
# =============================================================================
# withColumn: 새 컬럼 추가 또는 기존 컬럼 변환
# =============================================================================
print("=" * 60)
print("withColumn 변환 실습")
print("- 제곱값 계산")
print("- 상태 분류 (when/otherwise)")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

df_enhanced = (
    df_rate
    # 제곱값 계산
    .withColumn("squared", col("value") * col("value"))
    # 상태 분류
    .withColumn(
        "category",
        when(col("value") < 10, "small")
        .when(col("value") < 20, "medium")
        .otherwise("large")
    )
    # 필요한 컬럼만 선택
    .select("timestamp", "value", "squared", "category")
)

query = (
    df_enhanced.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(12)
query.stop()
print("\nwithColumn 실습 종료!")

### 3.4 스트림 변환 체이닝

In [ ]:
# =============================================================================
# 변환 체이닝: 여러 변환을 파이프라인으로 연결
# =============================================================================
print("=" * 60)
print("변환 체이닝 실습")
print("- 파생 컬럼 추가 → 필터링 → 선택")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

df_processed = (
    df_rate
    # 1. 파생 컬럼 추가
    .withColumn("is_even", col("value") % 2 == 0)
    .withColumn("tens", (col("value") / 10).cast("int"))
    # 2. 필터링
    .filter(col("is_even") == True)
    # 3. 필요한 컬럼만 선택
    .select("timestamp", "value", "tens")
)

query = (
    df_processed.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(12)
query.stop()
print("\n변환 체이닝 실습 종료!")

---

## Part 4: Output Mode

스트리밍 결과를 어떻게 출력할지 결정합니다.

| 모드 | 설명 | 사용 상황 |
|------|------|----------|
| `append` | 새로 추가된 행만 출력 | 집계 없는 단순 변환 |
| `update` | 변경된 행만 출력 | 집계 결과 중 업데이트된 것만 |
| `complete` | 전체 결과 출력 | 전체 집계 결과가 필요할 때 |

### Output Mode 선택 가이드

```
집계(groupBy)가 있나요?
│
├─ No → append 모드 (기본)
│
└─ Yes
    │
    ├─ 전체 결과가 필요한가?
    │   ├─ Yes → complete 모드
    │   └─ No → update 모드 (권장)
    │
    └─ 파일 저장이 목적인가?
        └─ Yes → append + Watermark
```

### 4.1 Append 모드

In [ ]:
# =============================================================================
# Append 모드: 새 행만 출력 (집계 없는 변환)
# =============================================================================
print("=" * 60)
print("Append 모드: 새 행만 출력")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()
df_filtered = df_rate.filter(col("value") > 5)

query = (
    df_filtered.writeStream
    .outputMode("append")
    .format("console")
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(10)
query.stop()
print("\nAppend 모드 실습 종료!")

### 4.2 Update 모드

**시나리오**: 상품별 주문 수를 실시간으로 모니터링합니다.
Update 모드는 **변경된 그룹만** 출력합니다.

In [ ]:
# =============================================================================
# Update 모드: 변경된 집계 결과만 출력
# =============================================================================
print("=" * 60)
print("Update 모드: 상품별 주문 수")
print("- 새 주문이 들어온 상품만 출력됩니다")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

# 시뮬레이션: value를 상품 코드로 변환
PRODUCTS = ["노트북", "스마트폰", "태블릿", "이어폰", "충전기"]

df_orders = (
    df_rate
    .withColumn("product",
        when(col("value") % 5 == 0, lit("노트북"))
        .when(col("value") % 5 == 1, lit("스마트폰"))
        .when(col("value") % 5 == 2, lit("태블릿"))
        .when(col("value") % 5 == 3, lit("이어폰"))
        .otherwise(lit("충전기"))
    )
    .groupBy("product")
    .agg(
        count("*").alias("order_count"),
        spark_sum("value").alias("total_value")
    )
)

query = (
    df_orders.writeStream
    .outputMode("update")  # 변경된 그룹만!
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(15)
query.stop()
print("\nUpdate 모드 실습 종료!")

**Update 모드 출력 예시**:
```
-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----------+-----------+
|product |order_count|total_value|
+--------+-----------+-----------+
|노트북  |3          |30         |   ← 새로 집계된 3개만 출력
|스마트폰|2          |17         |
+--------+-----------+-----------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------+-----------+-----------+
|product |order_count|total_value|
+--------+-----------+-----------+
|노트북  |5          |80         |   ← count가 3→5로 변경됨
|태블릿  |4          |52         |   ← 새로 변경된 그룹
+--------+-----------+-----------+
```

**핵심**: 이번 배치에서 **값이 변경된 그룹만** 출력됩니다!

### 4.3 Complete 모드

**시나리오**: 동일한 상품별 집계를 Complete 모드로 실행합니다.
Complete 모드는 **매번 전체 결과**를 출력합니다.

In [ ]:
# =============================================================================
# Complete 모드: 전체 집계 결과 출력
# =============================================================================
print("=" * 60)
print("Complete 모드: 매번 전체 상품 집계")
print("- 변경 유무와 관계없이 전체 출력")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

df_orders = (
    df_rate
    .withColumn("product",
        when(col("value") % 5 == 0, lit("노트북"))
        .when(col("value") % 5 == 1, lit("스마트폰"))
        .when(col("value") % 5 == 2, lit("태블릿"))
        .when(col("value") % 5 == 3, lit("이어폰"))
        .otherwise(lit("충전기"))
    )
    .groupBy("product")
    .agg(
        count("*").alias("order_count"),
        spark_sum("value").alias("total_value")
    )
)

query = (
    df_orders.writeStream
    .outputMode("complete")  # 전체 결과!
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(15)
query.stop()
print("\nComplete 모드 실습 종료!")

**Complete 모드 출력 예시**:
```
-------------------------------------------
Batch: 2
-------------------------------------------
+--------+-----------+-----------+
|product |order_count|total_value|
+--------+-----------+-----------+
|노트북  |5          |80         |   ← 전체 5개 상품 모두 출력
|스마트폰|4          |61         |
|태블릿  |4          |52         |
|이어폰  |3          |42         |
|충전기  |3          |39         |
+--------+-----------+-----------+
```

**핵심**: **항상 전체 집계 결과**가 출력됩니다!

---

### Update vs Complete 비교 정리

| 구분 | Update | Complete |
|------|--------|----------|
| 출력 내용 | 변경된 그룹만 | 전체 그룹 |
| 출력량 | 적음 | 많음 (그룹 수만큼) |
| 메모리 사용 | 낮음 | 높음 |
| 사용 상황 | 실시간 알림, 모니터링 | 대시보드, 전체 현황 |

---

## Part 5: Sink 패턴

스트리밍 결과를 어디로 출력할지 결정합니다.

| Sink | 용도 | Output Mode | 비고 |
|------|------|-------------|------|
| `console` | 개발/디버깅 | append, update, complete | 프로덕션 X |
| `file` | 데이터 저장 | append만 | 체크포인트 필수 |
| `kafka` | 이벤트 전달 | append, update | to_json 변환 필요 |
| `foreachBatch` | 커스텀 처리 | append, update, complete | 자유도 높음 |
| `memory` | 테스트 | append, complete | 임시 테이블로 쿼리 |

### 5.1 Console Sink (개발/디버깅)

In [ ]:
# =============================================================================
# Console Sink: 터미널에 결과 출력
# =============================================================================
print("=" * 60)
print("Console Sink: 디버깅용 출력")
print("- truncate=False: 긴 문자열 자르지 않음")
print("- numRows=20: 출력할 행 수")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

query = (
    df_rate.writeStream
    .format("console")
    .option("truncate", False)
    .option("numRows", 20)
    .outputMode("append")
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(10)
query.stop()
print("\nConsole Sink 실습 종료!")

### 5.2 foreachBatch Sink (커스텀 처리)

In [ ]:
# =============================================================================
# foreachBatch: 각 마이크로배치에 커스텀 로직 적용
# =============================================================================
print("=" * 60)
print("foreachBatch: 배치별 통계 계산")
print("=" * 60)

def process_batch(batch_df, batch_id):
    """각 마이크로배치 처리 함수"""
    count = batch_df.count()
    if count > 0:
        sum_val = batch_df.agg(spark_sum("value")).collect()[0][0]
        avg_val = batch_df.agg(avg("value")).collect()[0][0]
        print(f"배치 {batch_id}: {count}건, 합계={sum_val}, 평균={avg_val:.2f}")

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

query = (
    df_rate.writeStream
    .foreachBatch(process_batch)
    .trigger(processingTime="3 seconds")
    .start()
)

time.sleep(12)
query.stop()
print("\nforeachBatch 실습 종료!")

---

## Part 6: Trigger 설정

스트리밍 처리 주기를 결정합니다.

| Trigger | 설명 | 사용 상황 |
|---------|------|----------|
| `processingTime` | 고정 주기로 처리 | 일반적인 실시간 처리 |
| `once` | 한 번만 처리 후 종료 | 배치 스타일 처리 |
| `availableNow` | 현재까지 데이터 한 번 처리 | 백필(backfill) |
| `continuous` | 연속 처리 (밀리초 지연) | 초저지연 필요시 |

In [ ]:
# =============================================================================
# 다양한 Trigger 비교
# =============================================================================
print("=" * 60)
print("Trigger 비교")
print("- processingTime: 주기적 처리")
print("- once=True: 한 번만 처리")
print("=" * 60)

# processingTime Trigger
df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

query = (
    df_rate.writeStream
    .format("console")
    .trigger(processingTime="2 seconds")  # 2초마다 처리
    .start()
)

time.sleep(8)
query.stop()
print("\nTrigger 실습 종료!")

### Trigger 선택 가이드

| 상황 | 권장 Trigger | 주기 |
|------|-------------|------|
| 실시간 대시보드 | `processingTime` | 5~30초 |
| 준실시간 분석 | `processingTime` | 1~5분 |
| 알림/모니터링 | `processingTime` | 10~60초 |
| 스케줄링된 배치 | `once` | - |
| 대량 백필 | `availableNow` | - |

---

## Part 7: Tumbling Window 집계

시간 윈도우를 사용한 집계입니다. 스트리밍의 핵심 패턴입니다.

### Tumbling Window란?

**겹치지 않는 고정 크기 윈도우**입니다. 각 이벤트는 정확히 하나의 윈도우에 속합니다.

```
시간축: 0분 ──── 5분 ──── 10분 ──── 15분

윈도우1: [0분 ~ 5분)  ← 이벤트 A, B
윈도우2:       [5분 ~ 10분)  ← 이벤트 C, D
윈도우3:            [10분 ~ 15분)  ← 이벤트 E
```

In [ ]:
# =============================================================================
# Tumbling Window 집계
# =============================================================================
print("=" * 60)
print("Tumbling Window: 10초 윈도우")
print("- 10초 단위로 count, sum 집계")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

df_windowed = (
    df_rate
    .groupBy(
        window(col("timestamp"), "10 seconds")  # 10초 윈도우
    )
    .agg(
        count("*").alias("count"),
        spark_sum("value").alias("sum_value"),
        avg("value").alias("avg_value")
    )
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("count"),
        col("sum_value"),
        spark_round(col("avg_value"), 2).alias("avg_value")
    )
)

query = (
    df_windowed.writeStream
    .outputMode("update")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")
    .start()
)

time.sleep(25)
query.stop()
print("\nTumbling Window 실습 종료!")

---

## Part 8: Sliding Window 집계

**윈도우가 겹칩니다**. 하나의 이벤트가 여러 윈도우에 속할 수 있습니다.
이동 평균 계산 등에 활용됩니다.

```
윈도우 크기: 20초, 슬라이드 간격: 10초

시간축: 0초 ──── 10초 ──── 20초 ──── 30초

윈도우1: [0초 ──────────── 20초)
윈도우2:      [10초 ──────────── 30초)

이벤트 (15초): 윈도우1, 윈도우2 모두에 포함!
```

### Tumbling vs Sliding 비교

| 구분 | Tumbling | Sliding |
|------|----------|---------|
| 윈도우 겹침 | X | O |
| 이벤트 소속 | 1개 윈도우 | 여러 윈도우 |
| 문법 | `window(col, size)` | `window(col, size, slide)` |
| 용도 | 일반 집계 | 이동 평균, 트렌드 분석 |

In [ ]:
# =============================================================================
# Sliding Window: 이동 평균 계산
# =============================================================================
print("=" * 60)
print("Sliding Window: 20초 윈도우, 10초 슬라이드")
print("- 이동 평균 계산")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

df_sliding = (
    df_rate
    .groupBy(
        window(col("timestamp"), "20 seconds", "10 seconds")  # 윈도우 크기, 슬라이드 간격
    )
    .agg(
        count("*").alias("count"),
        avg("value").alias("moving_avg")
    )
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("count"),
        spark_round(col("moving_avg"), 2).alias("moving_avg")
    )
)

query = (
    df_sliding.writeStream
    .outputMode("update")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")
    .start()
)

time.sleep(35)
query.stop()
print("\nSliding Window 실습 종료!")

---

## Part 9: Watermark와 지연 데이터

### Watermark란?

실시간 시스템에서는 네트워크 지연 등으로 데이터가 늦게 도착할 수 있습니다.
**Watermark**는 "얼마나 늦은 데이터까지 기다릴 것인가"를 정의합니다.

```
현재 처리 시간: 10:15

[정상 데이터] 이벤트 시간 10:14 → 처리됨 ✓
[지연 데이터] 이벤트 시간 10:05 → 10분 지연! 처리할까?
```

**Watermark 동작 원리**:
```
Watermark = max(event_time) - 허용지연시간

예: max(event_time) = 10:15, 허용지연 = 10분
    Watermark = 10:05
    → 이벤트 시간 10:05 이전의 데이터는 무시됨 (윈도우 closed)
    → 이벤트 시간 10:05 이후의 데이터는 처리됨
```

### Watermark의 두 가지 역할

| 역할 | 설명 |
|------|------|
| **지연 데이터 처리** | 허용 시간 내 도착한 늦은 데이터 포함 |
| **상태 정리** | 오래된 윈도우 상태를 메모리에서 제거 |

### Watermark와 Output Mode

| Output Mode | Watermark 역할 | 출력 타이밍 |
|-------------|---------------|------------|
| `update` | 상태 정리용 (선택) | 매 배치마다 변경분 출력 |
| `append` | **필수** | 윈도우가 닫힌 후에만 출력 |
| `complete` | 상태 정리용 (선택) | 매 배치마다 전체 출력 |

### 9.1 Update 모드 + Watermark

**특징**: 윈도우가 닫히기 전에도 중간 결과를 볼 수 있습니다.
Watermark는 주로 **상태 정리**를 위해 사용됩니다.

In [ ]:
# =============================================================================
# Update 모드 + Watermark
# =============================================================================
print("=" * 60)
print("Update 모드 + Watermark (10초)")
print("- 매 배치마다 변경된 윈도우 출력")
print("- Watermark 지나면 윈도우 상태 정리")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

df_update_watermark = (
    df_rate
    .withWatermark("timestamp", "10 seconds")
    .groupBy(
        window(col("timestamp"), "10 seconds")
    )
    .agg(count("*").alias("count"))
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("count")
    )
)

query = (
    df_update_watermark.writeStream
    .outputMode("update")  # 변경된 윈도우만 출력
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")
    .start()
)

time.sleep(30)
query.stop()
print("\nUpdate + Watermark 실습 종료!")

**Update 모드 + Watermark 출력 예시**:
```
-------------------------------------------
Batch: 1 (시간: 10:00:05)
-------------------------------------------
+-------------------+-------------------+-----+
|window_start       |window_end         |count|
+-------------------+-------------------+-----+
|2026-01-21 10:00:00|2026-01-21 10:00:10|25   |  ← 아직 열린 윈도우
+-------------------+-------------------+-----+

-------------------------------------------
Batch: 2 (시간: 10:00:10)
-------------------------------------------
+-------------------+-------------------+-----+
|window_start       |window_end         |count|
+-------------------+-------------------+-----+
|2026-01-21 10:00:00|2026-01-21 10:00:10|50   |  ← count 증가 (25→50)
|2026-01-21 10:00:10|2026-01-21 10:00:20|15   |  ← 새 윈도우 시작
+-------------------+-------------------+-----+

-------------------------------------------
Batch: 3 (시간: 10:00:25) - Watermark = 10:00:15
-------------------------------------------
+-------------------+-------------------+-----+
|window_start       |window_end         |count|
+-------------------+-------------------+-----+
|2026-01-21 10:00:10|2026-01-21 10:00:20|50   |  ← 첫 번째 윈도우 사라짐 (closed)
|2026-01-21 10:00:20|2026-01-21 10:00:30|25   |
+-------------------+-------------------+-----+
```

**핵심 포인트**:
- 윈도우가 닫히기 **전**에도 중간 결과 출력
- Watermark를 지난 윈도우는 더 이상 출력되지 않음 (상태 정리됨)

### 9.2 Append 모드 + Watermark (필수!)

**특징**: 윈도우가 **확정된 후에만** 출력됩니다.
윈도우 집계에서 append 모드를 사용하려면 **Watermark 필수**입니다.

In [ ]:
# =============================================================================
# Append 모드 + Watermark (필수 조합)
# =============================================================================
print("=" * 60)
print("Append 모드 + Watermark")
print("- 윈도우가 완전히 닫힌 후에만 출력")
print("- 파일 저장에 적합 (중복 없음)")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

df_append_watermark = (
    df_rate
    .withWatermark("timestamp", "10 seconds")  # append 모드에서는 필수!
    .groupBy(
        window(col("timestamp"), "10 seconds")
    )
    .agg(count("*").alias("count"))
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("count")
    )
)

query = (
    df_append_watermark.writeStream
    .outputMode("append")  # 닫힌 윈도우만 출력
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")
    .start()
)

time.sleep(35)
query.stop()
print("\nAppend + Watermark 실습 종료!")

**Append 모드 + Watermark 출력 예시**:
```
-------------------------------------------
Batch: 1~3 (시간: 10:00:00 ~ 10:00:15)
-------------------------------------------
(출력 없음 - 아직 닫힌 윈도우가 없음)

-------------------------------------------
Batch: 4 (시간: 10:00:22) - Watermark = 10:00:12
-------------------------------------------
+-------------------+-------------------+-----+
|window_start       |window_end         |count|
+-------------------+-------------------+-----+
|2026-01-21 10:00:00|2026-01-21 10:00:10|50   |  ← 윈도우 닫힘! 한 번만 출력
+-------------------+-------------------+-----+

-------------------------------------------
Batch: 5 (시간: 10:00:32) - Watermark = 10:00:22
-------------------------------------------
+-------------------+-------------------+-----+
|window_start       |window_end         |count|
+-------------------+-------------------+-----+
|2026-01-21 10:00:10|2026-01-21 10:00:20|50   |  ← 다음 윈도우 닫힘
+-------------------+-------------------+-----+
```

**핵심 포인트**:
- 윈도우가 **완전히 닫힌 후** 한 번만 출력
- 출력이 **늦게** 나옴 (Watermark 대기 시간만큼)
- 파일 저장 시 **중복 없이** 안전하게 저장 가능

---

### Update vs Append 비교 (Watermark 있을 때)

| 구분 | Update | Append |
|------|--------|--------|
| 출력 타이밍 | 즉시 (중간 결과) | 윈도우 닫힌 후 |
| 같은 윈도우 출력 | 여러 번 (값 변경 시) | 한 번만 |
| 용도 | 실시간 모니터링 | 파일 저장, 정확한 결과 |
| Watermark | 선택 (상태 정리용) | **필수** |

### Watermark 설정 기준

| 상황 | 권장 Watermark | 이유 |
|------|---------------|------|
| 네트워크 안정적 | 1~5분 | 빠른 결과 출력 |
| 네트워크 불안정 | 10~30분 | 지연 데이터 처리 |
| 데이터 정확성 중요 | 30분~1시간 | 손실 최소화 |
| 빠른 응답 중요 | 1~2분 | 즉시 결과 |

---

## Part 10: 체크포인트와 장애 복구

스트리밍 애플리케이션의 상태를 저장하고 장애 시 복구합니다.

**체크포인트 저장 내용**:
- 현재 처리 오프셋 (어디까지 읽었는지)
- 집계 상태 (윈도우별 중간 결과)
- 메타데이터 (쿼리 설정 등)

**체크포인트 디렉토리 구조**:
```
/data/checkpoints/my-query/
├── commits/          # 커밋된 배치 정보
├── offsets/          # 소스 오프셋 (Kafka 등)
├── sources/          # 소스 메타데이터
├── state/            # 집계 상태
└── metadata          # 쿼리 메타데이터
```

In [ ]:
# =============================================================================
# 체크포인트 설정 예제
# =============================================================================
print("=" * 60)
print("체크포인트: /data/checkpoints/lesson")
print("=" * 60)

df_rate = spark.readStream.format("rate").option("rowsPerSecond", 5).load()

df_windowed = (
    df_rate
    .groupBy(window(col("timestamp"), "10 seconds"))
    .agg(count("*").alias("count"))
)

query = (
    df_windowed.writeStream
    .outputMode("update")
    .format("console")
    .option("checkpointLocation", "/data/checkpoints/lesson")  # 체크포인트!
    .trigger(processingTime="5 seconds")
    .start()
)

time.sleep(15)
query.stop()
print("\n체크포인트 실습 종료!")
print("체크포인트 저장됨: /data/checkpoints/lesson")

### 체크포인트 주의사항

| 주의사항 | 설명 |
|---------|------|
| 경로 고유성 | 쿼리마다 다른 체크포인트 경로 사용 |
| 삭제 금지 | 체크포인트 삭제 시 처음부터 재처리 |
| 스키마 변경 | 스키마 변경 시 새 체크포인트 필요 |
| 저장소 신뢰성 | HDFS, S3 등 신뢰할 수 있는 저장소 사용 |

---

## 실전 팁: 스트리밍 코드 개발 시 중간 결과 확인하기

Kafka 과제를 진행하기 전에, **스트리밍 코드를 단계별로 개발하는 방법**을 익혀봅시다.
한 번에 전체 코드를 작성하지 말고, **중간 결과를 확인**하며 진행하세요!

### 팁 1: DataFrame 타입과 스키마 확인

```python
# 스트리밍 DataFrame인지 확인
print(f"isStreaming: {df.isStreaming}")

# 스키마 확인 (컬럼명, 타입)
df.printSchema()
```

### 팁 2: 배치 DataFrame으로 먼저 테스트

스트리밍 코드를 작성하기 전에, **배치 모드**로 로직을 검증하세요.

```python
# 방법 1: 샘플 데이터로 배치 테스트
sample_data = [
    ('{"user_id": "U001", "amount": 100}',),
    ('{"user_id": "U002", "amount": 200}',),
]
df_batch = spark.createDataFrame(sample_data, ["value"])
df_parsed = df_batch.select(from_json(col("value"), schema).alias("data"))
df_parsed.show()  # 바로 결과 확인!

# 방법 2: Kafka에서 배치로 읽기 (startingOffsets + endingOffsets)
df_batch = (
    spark.read  # readStream이 아닌 read!
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "api-events")
    .option("startingOffsets", "earliest")
    .option("endingOffsets", "latest")
    .load()
)
df_batch.select(col("value").cast("string")).show(5, truncate=False)
```

### 팁 3: 단계별 스트리밍 개발 순서

```python
# Step 1: 원본 데이터 확인 (JSON 파싱 전)
query = (
    df_kafka
    .select(col("value").cast("string").alias("raw_json"))
    .writeStream.format("console").start()
)
# 출력 확인 후 query.stop()

# Step 2: JSON 파싱 결과 확인
query = (
    df_kafka
    .select(from_json(col("value").cast("string"), schema).alias("data"))
    .select("data.*")
    .writeStream.format("console").start()
)
# 모든 컬럼이 제대로 파싱되는지 확인

# Step 3: 변환 적용 후 확인
query = (
    df_parsed
    .filter(col("status_code") >= 400)
    .writeStream.format("console").start()
)

# Step 4: 최종 집계 추가
# ...
```

### 팁 4: 디버깅용 Console Sink 옵션

```python
.format("console")
.option("truncate", False)    # 긴 문자열 자르지 않음
.option("numRows", 50)        # 출력 행 수 늘리기
.trigger(processingTime="5 seconds")  # 빠른 피드백
```

### 팁 5: 스트리밍 쿼리 상태 확인

```python
# 쿼리 실행 중에
print(query.status)           # 현재 상태
print(query.lastProgress)     # 마지막 배치 정보
print(query.recentProgress)   # 최근 배치들 정보

# 에러 확인
print(query.exception())      # 예외 발생 시 확인
```

---

### 실전 예시: Kafka 데이터 단계별 확인

```python
# 1단계: Kafka 원본 확인
df_kafka = spark.readStream.format("kafka")...load()
df_kafka.select(col("value").cast("string")).writeStream.format("console").start()
# → {"user_id": "U001", "status_code": 200, ...} 확인

# 2단계: 스키마 정의 후 파싱
schema = StructType([...])
df_parsed = df_kafka.select(from_json(col("value").cast("string"), schema).alias("data"))
df_parsed.writeStream.format("console").start()
# → 모든 필드가 정상 파싱되는지 확인 (NULL이면 스키마 문제!)

# 3단계: 필드 펼치기
df_flat = df_parsed.select("data.*")
df_flat.printSchema()  # 스키마 확인
df_flat.writeStream.format("console").start()

# 4단계: 필터/변환 추가
df_filtered = df_flat.filter(col("status_code") >= 500)
df_filtered.writeStream.format("console").start()

# 5단계: 집계 추가 (최종)
df_agg = df_filtered.groupBy(window(...)).agg(...)
df_agg.writeStream.outputMode("update").format("console").start()
```

> **핵심**: 한 단계씩 확인하며 진행하면 에러 원인을 쉽게 찾을 수 있습니다!

---

## Part 11: Kafka 종합 실습

지금까지 배운 내용을 종합하여 실제 API 이벤트를 처리해봅시다.

> **실행 전**: Producer를 먼저 실행하세요!
> ```bash
> docker exec -it python-dev python producer.py --rate 10
> ```

In [ ]:
# =============================================================================
# 종합 실습: Kafka → 윈도우 집계 → Console
# =============================================================================
print("=" * 60)
print("종합 실습: Kafka API 이벤트 처리")
print("- 30초 윈도우로 endpoint별 통계")
print("- Producer 실행 필요: python producer.py --rate 10")
print("=" * 60)

# Kafka에서 읽기
df_kafka = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "api-events")
    .option("startingOffsets", "latest")
    .load()
)

# JSON 파싱
df_parsed = (
    df_kafka
    .select(from_json(col("value").cast("string"), event_schema).alias("data"))
    .select("data.*")
    .withColumn("event_time", to_timestamp(col("timestamp")))
)

# 30초 윈도우 집계
df_windowed = (
    df_parsed
    .withWatermark("event_time", "1 minute")
    .groupBy(
        window(col("event_time"), "30 seconds"),
        col("endpoint")
    )
    .agg(
        count("*").alias("request_count"),
        spark_round(avg("response_time_ms"), 2).alias("avg_latency"),
        spark_sum(when(col("status_code") >= 400, 1).otherwise(0)).alias("error_count")
    )
    .withColumn("error_rate", spark_round(col("error_count") / col("request_count") * 100, 1))
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("endpoint"),
        col("request_count"),
        col("avg_latency"),
        col("error_count"),
        col("error_rate")
    )
)

query = (
    df_windowed.writeStream
    .outputMode("update")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="10 seconds")
    .start()
)

time.sleep(45)
query.stop()
print("\nKafka 종합 실습 종료!")

---

## 퀴즈

**Q1. Structured Streaming에서 윈도우 집계 시 Watermark의 역할은?**

- A) 데이터 압축률을 높인다
- B) 지연 데이터를 얼마나 기다릴지 결정한다
- C) 출력 파일 크기를 제한한다
- D) 네트워크 대역폭을 조절한다

<details>
<summary>정답</summary>
B) 지연 데이터를 얼마나 기다릴지 결정한다
</details>

---

**Q2. 다음 중 append 모드에서 사용 가능한 쿼리는?**

- A) `df.groupBy("col").count()` (Watermark 없음)
- B) `df.filter(col("x") > 10)`
- C) `df.distinct()`
- D) `df.sort("col")`

<details>
<summary>정답</summary>
B) filter는 집계 없는 변환이므로 append 가능
</details>

---

**Q3. Sliding Window `window(col("time"), "10 minutes", "2 minutes")`의 의미는?**

- A) 10분 윈도우, 10분마다 새 윈도우
- B) 10분 윈도우, 2분마다 새 윈도우
- C) 2분 윈도우, 10분마다 새 윈도우
- D) 2분 윈도우, 2분마다 새 윈도우

<details>
<summary>정답</summary>
B) 10분 윈도우, 2분마다 새 윈도우 (겹침)
</details>

---

**Q4. Kafka 메시지를 Spark에서 처리할 때 JSON 파싱 순서는?**

- A) from_json → cast("string") → select("data.*")
- B) cast("string") → from_json → select("data.*")
- C) select("data.*") → cast("string") → from_json
- D) from_json → select("data.*") → cast("string")

<details>
<summary>정답</summary>
B) cast("string") → from_json → select("data.*")
</details>

---

**Q5. 체크포인트의 역할이 아닌 것은?**

- A) 장애 시 처리 상태 복구
- B) 중복 처리 방지
- C) 쿼리 성능 최적화
- D) 집계 중간 상태 저장

<details>
<summary>정답</summary>
C) 체크포인트는 성능 최적화가 아닌 장애 복구용
</details>

---

## 트러블슈팅 가이드

### 에러 1: Output Mode 불일치

**에러 메시지**:
```
AnalysisException: Append output mode not supported when there are streaming
aggregations on streaming DataFrames/DataSets without watermark
```

**원인**: 집계 쿼리에 append 모드를 사용했으나 Watermark가 없음

**해결**:
```python
# 방법 1: Watermark 추가
df.withWatermark("event_time", "10 minutes")
  .groupBy(window(...))
  .agg(...)
  .writeStream.outputMode("append")

# 방법 2: update 또는 complete 모드 사용
df.groupBy(...).agg(...)
  .writeStream.outputMode("update")  # Watermark 없이도 가능
```

---

### 에러 2: JSON 파싱 실패

**증상**: 모든 컬럼이 null로 나옴

**원인들**:
1. 스키마와 실제 JSON 구조 불일치
2. `cast("string")` 누락
3. 필드명 대소문자 불일치

**디버깅 방법**:
```python
# 1. 원본 데이터 확인 (바이너리)
df_kafka.select("value").writeStream.format("console").start()

# 2. 문자열 변환 후 확인
df_kafka.select(col("value").cast("string").alias("json_str")) \
  .writeStream.format("console").start()

# 3. 실제 JSON 구조 확인 후 스키마 수정
```

**해결**: 스키마를 실제 JSON과 일치시키기
```python
# 실제 JSON: {"user_id": "U001", "timestamp": "2024-01-01T10:00:00"}
schema = StructType([
    StructField("user_id", StringType(), True),     # 필드명 정확히
    StructField("timestamp", StringType(), True),   # 타입 맞추기
])
```

---

### 에러 3: Kafka 연결 실패

**에러 메시지**:
```
KafkaException: Failed to connect to kafka:9092
```

**해결**:
```bash
# 1. Kafka 상태 확인
docker compose ps

# 2. Kafka 로그 확인
docker logs kafka

# 3. 컨테이너 내부에서 연결 테스트
docker exec -it python-dev bash
nc -zv kafka 9092
```

---

### 에러 4: 체크포인트 충돌

**에러 메시지**:
```
Cannot update the query with the given checkpoint
StreamingQueryException: Cannot recover from a checkpoint with different query
```

**원인**: 쿼리 변경 후 기존 체크포인트 사용

**해결**:
```bash
# 체크포인트 삭제 후 재시작
rm -rf /data/checkpoints/query-name
```

---

### 에러 5: 윈도우 집계에서 데이터가 안 보임

**증상**: 윈도우 집계 결과가 출력되지 않음

**원인들**:
1. Watermark + append 모드: 윈도우가 닫힐 때까지 대기
2. Trigger 주기가 너무 김
3. 데이터 유입 없음

**해결**:
```python
# 방법 1: update 모드로 즉시 확인
.outputMode("update")

# 방법 2: Trigger 주기 단축
.trigger(processingTime="5 seconds")

# 방법 3: Watermark 지연 시간 단축 (테스트용)
.withWatermark("event_time", "30 seconds")
```

---

### 에러 6: 메모리 부족

**에러 메시지**:
```
java.lang.OutOfMemoryError: GC overhead limit exceeded
```

**해결**:
```python
# 1. 파티션 수 줄이기
.config("spark.sql.shuffle.partitions", 4)

# 2. 드라이버/Executor 메모리 증가
.config("spark.driver.memory", "2g")
.config("spark.executor.memory", "2g")

# 3. Watermark 설정으로 상태 정리 (집계 쿼리)
.withWatermark("event_time", "10 minutes")
```

---

### 빠른 디버깅 체크리스트

| 증상 | 확인 사항 |
|------|----------|
| 데이터 안 나옴 | Producer 실행 중? / Kafka 연결? |
| 모두 null | 스키마 일치? / cast("string")? |
| 집계 안 됨 | Output Mode? / Watermark? |
| 에러 메시지 | 로그 레벨을 DEBUG로 |
| OOM | 파티션 수 / Watermark |

---

## 과제: 실시간 API 모니터링

### 시나리오

운영팀에서 **실시간 API 모니터링**을 요청했습니다.
Kafka에서 API 이벤트를 수신하여 다양한 지표를 실시간으로 계산해야 합니다.

> **사전 준비**: Producer 실행
> ```bash
> docker exec -it python-dev python producer.py --rate 10
> ```

---

### 과제 1: 서버 에러 실시간 모니터링

**목표**: status_code가 500 이상인 서버 에러를 실시간으로 콘솔에 출력하세요.

**조건**:
- 출력 컬럼: event_time, endpoint, status_code, response_time_ms
- 5초마다 처리
- append 모드

<details>
<summary>힌트</summary>

- `filter(col("status_code") >= 500)`
- `outputMode("append")` (집계 없으므로)
- `trigger(processingTime="5 seconds")`

</details>

<details>
<summary>모범 답안</summary>

```python
df_kafka = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "api-events")
    .option("startingOffsets", "latest")
    .load()
)

df_parsed = (
    df_kafka
    .select(from_json(col("value").cast("string"), event_schema).alias("data"))
    .select("data.*")
    .withColumn("event_time", to_timestamp(col("timestamp")))
)

df_errors = (
    df_parsed
    .filter(col("status_code") >= 500)
    .select("event_time", "endpoint", "status_code", "response_time_ms")
)

query = (
    df_errors.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")
    .start()
)

query.awaitTermination()
```

</details>

---

### 과제 2: 5분 윈도우 집계 분석

**목표**: 5분 윈도우로 endpoint별 다음 지표를 계산하여 콘솔에 출력하세요.

**계산 지표**:
- total_requests: 총 요청 수
- avg_latency: 평균 응답시간
- error_count: 에러 수 (status_code >= 400)
- error_rate: 에러율 (%)

**조건**:
- Watermark: 2분 (테스트용으로 짧게)
- 30초마다 출력
- update 모드

<details>
<summary>힌트</summary>

- `withWatermark("event_time", "2 minutes")`
- `outputMode("update")` (콘솔 출력)
- `sum(when(..., 1).otherwise(0))`: 조건부 합계

</details>

<details>
<summary>모범 답안</summary>

```python
df_windowed = (
    df_parsed
    .withWatermark("event_time", "2 minutes")
    .groupBy(
        window(col("event_time"), "5 minutes"),
        col("endpoint")
    )
    .agg(
        count("*").alias("total_requests"),
        avg("response_time_ms").alias("avg_latency"),
        spark_sum(when(col("status_code") >= 400, 1).otherwise(0)).alias("error_count")
    )
    .withColumn("error_rate", col("error_count") / col("total_requests") * 100)
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("endpoint"),
        col("total_requests"),
        spark_round(col("avg_latency"), 2).alias("avg_latency"),
        col("error_count"),
        spark_round(col("error_rate"), 1).alias("error_rate")
    )
)

query = (
    df_windowed.writeStream
    .outputMode("update")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="30 seconds")
    .start()
)

query.awaitTermination()
```

</details>

---

### 과제 3: 응답시간 이동 평균 모니터링

**목표**: 10분 윈도우, 2분 슬라이드로 endpoint별 응답시간 이동 평균을 계산하세요.

**출력 지표**:
- moving_avg: 10분 이동 평균
- min_latency: 최소 응답시간
- max_latency: 최대 응답시간
- sample_count: 샘플 수

<details>
<summary>힌트</summary>

- Sliding Window: `window(col, "10 minutes", "2 minutes")`
- `outputMode("update")` (콘솔 출력)

</details>

<details>
<summary>모범 답안</summary>

```python
df_moving_avg = (
    df_parsed
    .withWatermark("event_time", "15 minutes")
    .groupBy(
        window(col("event_time"), "10 minutes", "2 minutes"),
        col("endpoint")
    )
    .agg(
        count("*").alias("sample_count"),
        avg("response_time_ms").alias("moving_avg"),
        min("response_time_ms").alias("min_latency"),
        max("response_time_ms").alias("max_latency")
    )
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("endpoint"),
        col("sample_count"),
        spark_round(col("moving_avg"), 2).alias("moving_avg"),
        col("min_latency"),
        col("max_latency")
    )
)

query = (
    df_moving_avg.writeStream
    .outputMode("update")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="30 seconds")
    .start()
)

query.awaitTermination()
```

</details>

---

### 과제 4: foreachBatch로 커스텀 처리

**목표**: foreachBatch를 사용하여 각 배치마다:
1. 배치 통계 출력 (총 건수, 에러 수)
2. 에러 비율이 30% 이상이면 경고 메시지 출력

<details>
<summary>힌트</summary>

- `foreachBatch(process_function)`
- `batch_df.count()`, `batch_df.filter(...).count()`

</details>

<details>
<summary>모범 답안</summary>

```python
def process_batch(batch_df, batch_id):
    """배치별 통계 및 경고"""
    if batch_df.isEmpty():
        print(f"배치 {batch_id}: 데이터 없음")
        return

    total = batch_df.count()
    errors = batch_df.filter(col("status_code") >= 400).count()
    error_rate = (errors / total) * 100 if total > 0 else 0

    print(f"=== 배치 {batch_id} ===")
    print(f"  총 요청: {total}")
    print(f"  에러: {errors}")
    print(f"  에러율: {error_rate:.1f}%")

    if error_rate >= 30:
        print(f"  [경고] 에러율이 {error_rate:.1f}%로 높습니다!")

query = (
    df_parsed.writeStream
    .foreachBatch(process_batch)
    .trigger(processingTime="10 seconds")
    .start()
)

query.awaitTermination()
```

</details>

---

### 보너스 과제: 에러 이벤트 Kafka 전달

**목표**: status_code >= 400인 에러 이벤트를 `error-alerts` 토픽으로 전송하세요.

**조건**:
- key: endpoint
- value: 전체 이벤트 JSON
- 체크포인트: `/data/checkpoints/error-alerts`

<details>
<summary>힌트</summary>

- `to_json(struct("*"))`: 모든 컬럼을 JSON으로
- `format("kafka")`
- `option("topic", "error-alerts")`

</details>

<details>
<summary>모범 답안</summary>

```python
df_error_events = (
    df_parsed
    .filter(col("status_code") >= 400)
    .select(
        col("endpoint").alias("key"),
        to_json(struct(
            col("event_time"),
            col("request_id"),
            col("endpoint"),
            col("status_code"),
            col("response_time_ms")
        )).alias("value")
    )
)

query = (
    df_error_events.writeStream
    .outputMode("append")
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("topic", "error-alerts")
    .option("checkpointLocation", "/data/checkpoints/error-alerts")
    .trigger(processingTime="10 seconds")
    .start()
)

query.awaitTermination()
```

</details>

---

## 핵심 요약

### 스트리밍 소스
```python
# Kafka (프로덕션)
spark.readStream.format("kafka").option("subscribe", "topic").load()

# Rate (테스트)
spark.readStream.format("rate").option("rowsPerSecond", 10).load()
```

### JSON 파싱
```python
df.select(from_json(col("value").cast("string"), schema).alias("data"))
  .select("data.*")
```

### Output Mode
| 모드 | 용도 |
|------|------|
| `append` | 집계 없는 변환, 파일 저장 |
| `update` | 집계 결과 변경분 |
| `complete` | 전체 집계 결과 |

### 윈도우 집계
```python
# Tumbling (겹치지 않음)
window(col("time"), "5 minutes")

# Sliding (겹침)
window(col("time"), "10 minutes", "2 minutes")
```

### Watermark
```python
df.withWatermark("event_time", "10 minutes")
  .groupBy(window(...))
  .agg(...)
```

### 체크포인트
```python
.option("checkpointLocation", "/path/to/checkpoint")
```

---

## 세션 종료

In [ ]:
spark.stop()
print("SparkSession 종료 완료!")
print("\n수업을 마칩니다. 수고하셨습니다!")